In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [2]:
nodes = pd.read_csv('../data/nodes.csv')
edges = pd.read_csv('../data/edges.csv')

# remove : character from column names to make them queryable
nodes = nodes.rename(columns={':ID': 'ID', ':LABEL': 'LABEL'})
edges = edges.rename(columns={':START_ID': 'START_ID', ':END_ID': 'END_ID', ':TYPE': 'TYPE'})

In [3]:
nodes.head()

,ID,name,LABEL
0,C0038195,State Medicine,Activities & Behaviors
1,C1290952,Taking medication,Activities & Behaviors
2,C0680552,legalization,Activities & Behaviors
3,C0085092,Parenting behavior,Activities & Behaviors
4,C1096771,Murderer,Activities & Behaviors


In [4]:
edges.head()

,START_ID,END_ID,TYPE,pmids
0,C1273870,C0282623,ADMINISTERED_TO_ABatAB,1
1,C1273870,C1138603,ADMINISTERED_TO_ABatCI,2
2,C0035028,C0221192,ADMINISTERED_TO_ABatCI,2
3,C1273870,C0221192,ADMINISTERED_TO_ABatCI,53
4,C0556656,C0221192,ADMINISTERED_TO_ABatCI,7


In [5]:
print(edges.shape[0])
edges = edges.drop_duplicates()
edges = edges.dropna()
edges.shape

20783717


(20783717, 4)

# Look at the Metagraph to see if some edges can be combined

In [6]:
def list_connections(metanode1, metanode2):
    
    # get the ids for the given node
    ids1 = list(nodes.query("LABEL == @metanode1")['ID'])
    
    # if looking at edges between the same metanode, can get edges right away
    if metanode1 == metanode2:
        res = edges.query('START_ID in @ids1 and END_ID in @ids1')
    
    else:
        # Ids for second metanode
        ids2 = list(nodes.query("LABEL == @metanode2")['ID'])
    
        # 
        res1 = edges.query('START_ID in @ids1 and END_ID in @ids2')
        res2 = edges.query('START_ID in @ids2 and END_ID in @ids1')
    
        res = pd.concat([res1, res2])
    
    return res['TYPE'].value_counts()
    

In [7]:
id_to_type = nodes.set_index('ID')['LABEL'].to_dict()

In [8]:
combos = []
for row in edges.drop_duplicates(subset='TYPE').itertuples(index=False):
    combos.append((id_to_type[row[0]], id_to_type[row[1]]))
combos = [tuple(sorted(c)) for c in combos]
combos = sorted(list(set(combos)))
len(combos)

116

In [9]:
combos

[('Activities & Behaviors', 'Activities & Behaviors'),
 ('Activities & Behaviors', 'Anatomy'),
 ('Activities & Behaviors', 'Chemicals & Drugs'),
 ('Activities & Behaviors', 'Concepts & Ideas'),
 ('Activities & Behaviors', 'Devices'),
 ('Activities & Behaviors', 'Disorders'),
 ('Activities & Behaviors', 'Genes & Molecular Sequences'),
 ('Activities & Behaviors', 'Living Beings'),
 ('Activities & Behaviors', 'Objects'),
 ('Activities & Behaviors', 'Occupations'),
 ('Activities & Behaviors', 'Organizations'),
 ('Activities & Behaviors', 'Phenomena'),
 ('Activities & Behaviors', 'Physiology'),
 ('Activities & Behaviors', 'Procedures'),
 ('Anatomy', 'Anatomy'),
 ('Anatomy', 'Chemicals & Drugs'),
 ('Anatomy', 'Concepts & Ideas'),
 ('Anatomy', 'Devices'),
 ('Anatomy', 'Disorders'),
 ('Anatomy', 'Genes & Molecular Sequences'),
 ('Anatomy', 'Geographic Areas'),
 ('Anatomy', 'Living Beings'),
 ('Anatomy', 'Objects'),
 ('Anatomy', 'Occupations'),
 ('Anatomy', 'Organizations'),
 ('Anatomy', 'Pheno

In [10]:
if os.path.exists("../data/edge_pair_counts_orig.pkl"):
    combo_counts = pickle.load(open( "../data/edge_pair_counts_orig.pkl", "rb" ))
else:
    # This block took over 7 hours to run, pickleing for future use
    combo_counts = dict()
    for comb in tqdm(combos):
        combo_counts[comb] = list_connections(comb[0], comb[1])
    pickle.dump(combo_counts, open( "../data/edge_pair_counts_orig.pkl", "wb" ) )

In [11]:
edge_numbs = {k: len(v) for k, v in combo_counts.items()}

In [12]:
most_to_least = sorted(list(edge_numbs.items()), key = lambda x: x[1], reverse=True)
for item in most_to_least:
    print(item[0], ':', item[1])

('Chemicals & Drugs', 'Disorders') : 56
('Chemicals & Drugs', 'Procedures') : 55
('Chemicals & Drugs', 'Concepts & Ideas') : 54
('Disorders', 'Procedures') : 53
('Chemicals & Drugs', 'Physiology') : 53
('Disorders', 'Genes & Molecular Sequences') : 52
('Chemicals & Drugs', 'Genes & Molecular Sequences') : 52
('Physiology', 'Procedures') : 52
('Genes & Molecular Sequences', 'Physiology') : 52
('Genes & Molecular Sequences', 'Procedures') : 52
('Disorders', 'Physiology') : 51
('Concepts & Ideas', 'Genes & Molecular Sequences') : 50
('Chemicals & Drugs', 'Living Beings') : 49
('Concepts & Ideas', 'Disorders') : 49
('Chemicals & Drugs', 'Phenomena') : 48
('Disorders', 'Disorders') : 48
('Disorders', 'Living Beings') : 46
('Concepts & Ideas', 'Procedures') : 46
('Chemicals & Drugs', 'Chemicals & Drugs') : 46
('Concepts & Ideas', 'Physiology') : 45
('Anatomy', 'Chemicals & Drugs') : 45
('Genes & Molecular Sequences', 'Living Beings') : 43
('Anatomy', 'Procedures') : 42
('Activities & Behavio

In [13]:
def change_edge_type(from_type, to_type, swap=False):
    idx = edges.query('TYPE == @from_type').index
    edges.loc[idx, 'TYPE'] = to_type
    if swap:
        tmp = edges.loc[idx, 'START_ID']
        edges.loc[idx, 'START_ID'] = edges.loc[idx, 'END_ID']
        edges.loc[idx, 'END_ID'] = tmp
                                             
def merge_edge_types(from_list, to_type, swap=False):
    for from_type in from_list:
        change_edge_type(from_type, to_type, swap=swap)
        
def drop_edges_from_list(drop_edges):
    idx = edges.query('TYPE in @drop_edges').index
    edges.drop(idx, inplace=True)

In [14]:
combo_num = 0

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Activities & Behaviors 

AFFECTS_ABafAB                   2467
ISA_ABi>AB                       1738
NEG_AFFECTS_ABnafAB               226
METHOD_OF_ABmoAB                  147
LOCATION_OF_ABloAB                100
PREDISPOSES_ABpsAB                 37
COEXISTS_WITH_ABcwAB               13
STIMULATES_ABstAB                  12
TREATS_ABtAB                       12
MANIFESTATION_OF_ABmfo>AB          10
CAUSES_ABcAB                       10
PROCESS_OF_ABpro>AB                 6
PREVENTS_ABpvAB                     4
USES_ABuAB                          4
compared_with_ABcpwAB               3
NEG_PREDISPOSES_ABnpsAB             3
higher_than_ABhtAB                  3
DIAGNOSES_ABdgAB                    2
PRECEDES_ABpc>AB                    2
lower_than_ABltAB                   2
NEG_LOCATION_OF_ABnloAB             2
same_as_ABsaAB                      1
ADMINISTERED_TO_ABatAB              1
NEG_COEXISTS_WITH_ABncwAB           1
different_from_ABdfAB               

In [15]:
# Small number of total relations, will push all edges that would modulate to AFFECTS edge 
to_merge = ['AFFECTS_ABafAB', 'PREDISPOSES_ABpsAB', 'STIMULATES_ABstAB', 'TREATS_ABtAB', 'CAUSES_ABcAB', 'PREVENTS_ABpvAB']
merge_edge_types(to_merge, 'AFFECTS_ABafAB')

In [16]:
combo_num = 1

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Anatomy 

LOCATION_OF_AloAB         1202
AFFECTS_AafAB               80
ASSOCIATED_WITH_AawAB       31
PRODUCES_ABpd>A             28
NEG_LOCATION_OF_AnloAB      23
TREATS_ABtA                  9
NEG_AFFECTS_AnafAB           9
CAUSES_AcAB                  5
METHOD_OF_ABmoA              4
COEXISTS_WITH_ABcwA          4
DISRUPTS_AdsAB               4
NEG_PRODUCES_ABnpdA          2
PROCESS_OF_Apro>AB           2
USES_ABuA                    2
PREDISPOSES_ABpsA            2
PREVENTS_ABpvA               1
STIMULATES_ABstA             1
PRECEDES_ABpc>A              1
AUGMENTS_ABagA               1
DIAGNOSES_ABdgA              1
NEG_PREVENTS_ABnpvA          1
PART_OF_Apo>AB               1
NEG_TREATS_ABntA             1
Name: TYPE, dtype: int64


Small counts here, No need to do anything, these will get dropped

In [17]:
combo_num = 2

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Chemicals & Drugs 

AFFECTS_CDafAB                 3662
USES_ABuCD                     3637
AUGMENTS_CDagAB                2217
CAUSES_CDcAB                   1139
DISRUPTS_CDdsAB                1118
TREATS_CDtAB                    596
ASSOCIATED_WITH_CDawAB          554
NEG_AFFECTS_CDnafAB             521
PRODUCES_ABpd>CD                420
compared_with_ABcpwCD           350
PREDISPOSES_CDpsAB              249
PREVENTS_CDpvAB                 233
INHIBITS_CDinAB                 232
COEXISTS_WITH_CDcwAB            146
METHOD_OF_ABmoCD                117
STIMULATES_ABstCD               116
NEG_AUGMENTS_CDnagAB            108
higher_than_ABhtCD              100
NEG_CAUSES_CDncAB                65
NEG_DISRUPTS_CDndsAB             56
NEG_USES_ABnuCD                  51
NEG_ASSOCIATED_WITH_CDnawAB      26
COMPLICATES_CDcpAB               24
NEG_TREATS_CDntAB                20
PRECEDES_CDpc>AB                 19
lower_than_ABltCD                18
NEG_INHIBITS_CDni

In [18]:
# Small numbers again, will merge to 1 type. AFFECTS for anyhthing would modulate

to_merge = ['AFFECTS_CDafAB', 'AUGMENTS_CDagAB', 'CAUSES_CDcAB',
'DISRUPTS_CDdsAB', 'TREATS_CDtAB', 'ASSOCIATED_WITH_CDawAB',
'PREDISPOSES_CDpsAB', 'PREVENTS_CDpvAB', 'INHIBITS_CDinAB',
'COMPLICATES_CDcpAB']

merge_edge_types(to_merge, 'AFFECTS_CDafAB')

In [19]:
combo_num = 3

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Concepts & Ideas 

ISA_CIi>AB                       302
PROCESS_OF_ABpro>CI              195
AFFECTS_CIafAB                   189
TREATS_ABtCI                     155
METHOD_OF_CImoAB                  84
ISA_ABi>CI                        59
USES_ABuCI                        47
CAUSES_ABcCI                      42
OCCURS_IN_ABoiCI                  42
PREVENTS_ABpvCI                   34
NEG_AFFECTS_CInafAB               33
LOCATION_OF_CIloAB                32
COEXISTS_WITH_ABcwCI              20
DIAGNOSES_ABdgCI                  15
PREDISPOSES_CIpsAB                11
NEG_TREATS_ABntCI                  9
PROCESS_OF_CIpro>AB                8
ADMINISTERED_TO_ABatCI             6
STIMULATES_ABstCI                  6
NEG_PROCESS_OF_ABnproCI            6
MANIFESTATION_OF_CImfo>AB          4
NEG_OCCURS_IN_ABnoiCI              4
PRECEDES_CIpc>AB                   3
AUGMENTS_CIagAB                    2
MANIFESTATION_OF_ABmfo>CI          2
PRECEDES_ABpc>CI             

All way to small in number to ever make it past filtering

In [20]:
combo_num = 4

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Devices 

USES_ABuDV               493
TREATS_DVtAB              94
compared_with_DVcpwAB     22
METHOD_OF_ABmoDV          18
COEXISTS_WITH_DVcwAB      15
NEG_USES_ABnuDV           11
CAUSES_DVcAB              10
PREVENTS_DVpvAB            9
higher_than_DVhtAB         6
LOCATION_OF_DVloAB         3
STIMULATES_DVstAB          2
AFFECTS_DVafAB             2
NEG_PREVENTS_DVnpvAB       1
PROCESS_OF_ABpro>DV        1
NEG_METHOD_OF_DVnmoAB      1
NEG_CAUSES_DVncAB          1
Name: TYPE, dtype: int64


In [21]:
combo_num = 5

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Disorders 

TREATS_ABtDO                     14003
PREDISPOSES_ABpsDO                5493
AFFECTS_DOafAB                    4329
PREVENTS_ABpvDO                   3315
CAUSES_ABcDO                      2176
COEXISTS_WITH_ABcwDO              1255
DISRUPTS_DOdsAB                    968
ASSOCIATED_WITH_ABawDO             784
DIAGNOSES_ABdgDO                   651
NEG_TREATS_ABntDO                  532
PROCESS_OF_DOpro>AB                336
NEG_AFFECTS_ABnafDO                319
NEG_PREDISPOSES_DOnpsAB            289
MANIFESTATION_OF_ABmfo>DO          216
PROCESS_OF_ABpro>DO                132
MANIFESTATION_OF_DOmfo>AB          128
NEG_PREVENTS_ABnpvDO               120
ISA_DOi>AB                         108
ISA_ABi>DO                          78
NEG_CAUSES_ABncDO                   72
NEG_COEXISTS_WITH_DOncwAB           52
PRECEDES_DOpc>AB                    32
METHOD_OF_ABmoDO                    28
NEG_DISRUPTS_DOndsAB                27
NEG_ASSOCIATED_WITH_ABnaw

Larger numbers here, will try for negative and positive regulation

In [22]:
# AB that lessens DO
merge_edge_types(['TREATS_ABtDO',  'PREVENTS_ABpvDO', 'DIAGNOSES_ABdgDO'], 'TREATS_ABtDO')
# AB that increases or assoc with DO
merge_edge_types(['PREDISPOSES_ABpsDO', 'CAUSES_ABcDO',  
                  'COEXISTS_WITH_ABcwDO', 'ASSOCIATED_WITH_ABawDO'], 'ASSOCIATED_WITH_ABawDO')

# DO to AB edges
merge_edge_types(['AFFECTS_DOafAB', 'DISRUPTS_DOdsAB'], 'AFFECTS_DOafAB')



In [23]:
combo_num = 6

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Genes & Molecular Sequences 

AFFECTS_GafAB                 1151
USES_ABuG                      668
ASSOCIATED_WITH_GawAB          598
AUGMENTS_GagAB                 588
CAUSES_GcAB                    417
PREDISPOSES_GpsAB              319
DISRUPTS_GdsAB                 220
TREATS_ABtG                    167
PRODUCES_ABpd>G                115
compared_with_GcpwAB            77
NEG_AFFECTS_GnafAB              72
PREVENTS_GpvAB                  35
NEG_ASSOCIATED_WITH_GnawAB      22
higher_than_GhtAB               21
COEXISTS_WITH_GcwAB             18
NEG_AUGMENTS_GnagAB             18
NEG_CAUSES_GncAB                14
METHOD_OF_GmoAB                 12
INHIBITS_GinAB                  10
STIMULATES_GstAB                 9
NEG_USES_ABnuG                   8
LOCATION_OF_GloAB                5
lower_than_GltAB                 3
NEG_PREDISPOSES_GnpsAB           3
NEG_DISRUPTS_GndsAB              3
NEG_STIMULATES_GnstAB            2
same_as_ABsaG                    

All too small to matter

In [24]:
combo_num = 7

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Living Beings 

PROCESS_OF_ABpro>LB            20632
TREATS_ABtLB                    4972
NEG_PROCESS_OF_ABnproLB          655
AFFECTS_ABafLB                   483
ADMINISTERED_TO_ABatLB           400
USES_LBuAB                       280
NEG_TREATS_ABntLB                233
OCCURS_IN_ABoiLB                  79
DIAGNOSES_LBdgAB                  59
NEG_AFFECTS_ABnafLB               37
CAUSES_LBcAB                      35
NEG_ADMINISTERED_TO_ABnatLB       29
METHOD_OF_ABmoLB                  28
LOCATION_OF_LBloAB                26
PRODUCES_ABpd>LB                   6
NEG_USES_LBnuAB                    4
INTERACTS_WITH_LBiwAB              4
NEG_CAUSES_LBncAB                  3
PRODUCES_LBpd>AB                   3
NEG_OCCURS_IN_ABnoiLB              2
PREVENTS_ABpvLB                    2
PROCESS_OF_LBpro>AB                1
DISRUPTS_LBdsAB                    1
NEG_ASSOCIATED_WITH_LBnawAB        1
Name: TYPE, dtype: int64


'process_of' here is the only potentially useable edge, but we will make undirected

In [25]:
merge_edge_types(['PROCESS_OF_ABpro>LB', 'OCCURS_IN_ABoiLB'], 'OCCURS_IN_ABoiLB')
change_edge_type('USES_LBuAB', 'OCCURS_IN_ABoiLB', swap=True)

In [26]:
combo_num = 8

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Objects 

USES_ABuOB                 81
LOCATION_OF_OBloAB         62
AFFECTS_OBafAB             60
CAUSES_OBcAB               11
TREATS_OBtAB               10
NEG_AFFECTS_OBnafAB         7
METHOD_OF_OBmoAB            3
compared_with_ABcpwOB       3
ISA_OBi>AB                  3
PREDISPOSES_OBpsAB          3
NEG_USES_ABnuOB             2
AUGMENTS_OBagAB             2
ASSOCIATED_WITH_OBawAB      1
NEG_LOCATION_OF_OBnloAB     1
PART_OF_OBpo>AB             1
same_as_ABsaOB              1
PROCESS_OF_ABpro>OB         1
STIMULATES_OBstAB           1
ISA_ABi>OB                  1
PRODUCES_ABpd>OB            1
Name: TYPE, dtype: int64


Too small to use

In [27]:
combo_num = 9

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Occupations 

METHOD_OF_ABmoOC          1911
ISA_ABi>OC                  25
NEG_METHOD_OF_ABnmoOC       20
PROCESS_OF_ABpro>OC         18
COEXISTS_WITH_ABcwOC         5
TREATS_ABtOC                 4
ISA_OCi>AB                   3
PREVENTS_OCpvAB              1
LOCATION_OF_ABloOC           1
ADMINISTERED_TO_ABatOC       1
PREDISPOSES_OCpsAB           1
Name: TYPE, dtype: int64


Too small

In [28]:
combo_num = 10

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Organizations 

LOCATION_OF_ORloAB           6786
NEG_LOCATION_OF_ORnloAB        85
ISA_ORi>AB                     55
AFFECTS_ABafOR                 40
ISA_ABi>OR                      6
NEG_AFFECTS_ABnafOR             4
PROCESS_OF_ABpro>OR             3
PRODUCES_ORpd>AB                2
COEXISTS_WITH_ORcwAB            2
TREATS_ABtOR                    2
METHOD_OF_ABmoOR                1
NEG_COEXISTS_WITH_ORncwAB       1
Name: TYPE, dtype: int64


Too small

In [29]:
combo_num = 11

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Phenomena 

AFFECTS_PHafAB               288
PROCESS_OF_ABpro>PH          188
PROCESS_OF_PHpro>AB           94
TREATS_ABtPH                  48
NEG_AFFECTS_ABnafPH           28
NEG_PROCESS_OF_PHnproAB       14
CAUSES_ABcPH                  11
METHOD_OF_PHmoAB              10
COEXISTS_WITH_ABcwPH           9
ISA_PHi>AB                     6
ISA_ABi>PH                     5
DIAGNOSES_ABdgPH               5
PRECEDES_ABpc>PH               4
PREDISPOSES_PHpsAB             3
MANIFESTATION_OF_PHmfo>AB      3
PREVENTS_ABpvPH                3
NEG_TREATS_ABntPH              2
DISRUPTS_PHdsAB                2
STIMULATES_ABstPH              1
LOCATION_OF_PHloAB             1
NEG_COEXISTS_WITH_ABncwPH      1
Name: TYPE, dtype: int64


In [30]:
combo_num = 12

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Physiology 

AFFECTS_ABafPS               1469
COEXISTS_WITH_PScwAB          836
PROCESS_OF_ABpro>PS           582
TREATS_ABtPS                  210
PROCESS_OF_PSpro>AB           195
NEG_AFFECTS_PSnafAB           177
PRECEDES_ABpc>PS              157
CAUSES_PScAB                  107
PRECEDES_PSpc>AB              100
NEG_PROCESS_OF_ABnproPS        52
NEG_COEXISTS_WITH_PSncwAB      51
DISRUPTS_ABdsPS                44
STIMULATES_ABstPS              34
PREVENTS_ABpvPS                32
ISA_PSi>AB                     29
ISA_ABi>PS                     25
PREDISPOSES_ABpsPS             22
NEG_TREATS_ABntPS              19
USES_PSuAB                     14
METHOD_OF_ABmoPS               11
MANIFESTATION_OF_ABmfo>PS      10
MANIFESTATION_OF_PSmfo>AB       7
NEG_CAUSES_PSncAB               5
DIAGNOSES_ABdgPS                4
NEG_DISRUPTS_ABndsPS            4
NEG_PRECEDES_ABnpcPS            3
NEG_PREDISPOSES_PSnpsAB         2
OCCURS_IN_ABoiPS                2
ASSOCIAT

In [31]:
combo_num = 13

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Activities & Behaviors --- Procedures 

METHOD_OF_ABmoPR             4368
COEXISTS_WITH_ABcwPR         2112
AFFECTS_PRafAB               1142
STIMULATES_ABstPR             994
USES_ABuPR                    797
ISA_PRi>AB                    663
TREATS_PRtAB                  300
LOCATION_OF_PRloAB            224
PRECEDES_PRpc>AB              202
NEG_AFFECTS_PRnafAB           164
DIAGNOSES_PRdgAB              163
compared_with_ABcpwPR         121
NEG_METHOD_OF_PRnmoAB         100
PREVENTS_PRpvAB                89
NEG_COEXISTS_WITH_PRncwAB      62
NEG_STIMULATES_PRnstAB         54
ISA_ABi>PR                     37
higher_than_PRhtAB             30
PRECEDES_ABpc>PR               30
NEG_USES_ABnuPR                21
PROCESS_OF_ABpro>PR            14
CAUSES_ABcPR                   14
COMPLICATES_PRcpAB             13
NEG_TREATS_PRntAB              11
lower_than_PRltAB               6
NEG_LOCATION_OF_ABnloPR         5
NEG_DIAGNOSES_PRndgAB           4
NEG_PREVENTS_PRnpvAB            4
NEG_high

In [32]:
combo_num = 14

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Anatomy 

PART_OF_Apo>A              202994
LOCATION_OF_AloA            72116
NEG_PART_OF_AnpoA            3290
PRODUCES_Apd>A               1860
NEG_LOCATION_OF_AnloA        1827
AFFECTS_AafA                  142
COEXISTS_WITH_AcwA            138
NEG_PRODUCES_AnpdA             63
AUGMENTS_AagA                  36
INHIBITS_AinA                  20
ASSOCIATED_WITH_AawA           19
DISRUPTS_AdsA                  15
NEG_AFFECTS_AnafA               8
compared_with_AcpwA             7
INTERACTS_WITH_AiwA             5
STIMULATES_AstA                 4
higher_than_AhtA                3
CONVERTS_TO_Act>A               3
NEG_COEXISTS_WITH_AncwA         2
ADMINISTERED_TO_AatA            2
CAUSES_AcA                      2
lower_than_AltA                 1
PROCESS_OF_Apro>A               1
NEG_AUGMENTS_AnagA              1
same_as_AsaA                    1
different_from_AdfA             1
Name: TYPE, dtype: int64


In [33]:
merge_edge_types(['PART_OF_Apo>A', 'LOCATION_OF_AloA'], 'LOCATION_OF_AloA')

In [34]:
combo_num = 15

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Chemicals & Drugs 

LOCATION_OF_AloCD             860360
PART_OF_CDpo>A                325076
AFFECTS_CDafA                 180888
AUGMENTS_CDagA                157053
PRODUCES_Apd>CD               108397
DISRUPTS_CDdsA                 76820
NEG_LOCATION_OF_AnloCD         21307
NEG_AFFECTS_CDnafA             15580
ADMINISTERED_TO_CDatA          12451
NEG_PART_OF_CDnpoA              6877
NEG_PRODUCES_AnpdCD             6827
NEG_AUGMENTS_CDnagA             5232
PRODUCES_CDpd>A                 3913
NEG_DISRUPTS_CDndsA             2937
COEXISTS_WITH_AcwCD              642
PART_OF_Apo>CD                   334
INTERACTS_WITH_AiwCD             324
TREATS_CDtA                      278
CAUSES_CDcA                      264
ASSOCIATED_WITH_AawCD            205
NEG_ADMINISTERED_TO_CDnatA       163
STIMULATES_CDstA                 128
USES_AuCD                        108
INHIBITS_AinCD                   102
compared_with_CDcpwA              50
higher_than_AhtCD                 12
NEG_IN

We will go for two distinct types.  

1. Location where CD naturally exists
2. Location where CD has an effect

In [35]:
# Change to a localization of compound
to_merge = ['LOCATION_OF_AloCD', 'PRODUCES_Apd>CD','COEXISTS_WITH_AcwCD']
merge_edge_types(to_merge, 'LOCATION_OF_AloCD')
merge_edge_types(['PRODUCES_CDpd>A','PART_OF_CDpo>A'], 'LOCATION_OF_AloCD', swap=True)
 
# Change to localization of compound's affect
to_merge = ['AFFECTS_CDafA', 'AUGMENTS_CDagA',  'DISRUPTS_CDdsA', 'ADMINISTERED_TO_CDatA',  'TREATS_CDtA']
merge_edge_types(to_merge, 'AFFECTS_CDafA')

In [36]:
combo_num = 16

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Concepts & Ideas 

LOCATION_OF_AloCI             33058
NEG_LOCATION_OF_CInloA         1196
PART_OF_CIpo>A                 1074
PART_OF_Apo>CI                 1025
AFFECTS_CIafA                   193
ASSOCIATED_WITH_AawCI           146
PROCESS_OF_CIpro>A               69
AUGMENTS_CIagA                   47
PRODUCES_Apd>CI                  44
NEG_PART_OF_AnpoCI               26
CAUSES_AcCI                      24
DISRUPTS_CIdsA                   24
PRODUCES_CIpd>A                  22
NEG_AFFECTS_CInafA               14
OCCURS_IN_AoiCI                  13
COEXISTS_WITH_AcwCI              13
NEG_ASSOCIATED_WITH_AnawCI       12
PROCESS_OF_Apro>CI                9
ADMINISTERED_TO_CIatA             7
ISA_Ai>CI                         7
USES_CIuA                         6
TREATS_AtCI                       6
DIAGNOSES_CIdgA                   5
NEG_CAUSES_AncCI                  2
compared_with_AcpwCI              2
NEG_PRODUCES_AnpdCI               1
PRECEDES_Apc>CI                  

Only 1 true relation here:  Location_of

In [37]:
to_merge = ['LOCATION_OF_AloCI', 'ASSOCIATED_WITH_AawCI','PRODUCES_Apd>CI', 'PART_OF_Apo>CI']
merge_edge_types(to_merge, 'LOCATION_OF_AloCI')       

to_merge = ['AFFECTS_CIafA', 'PART_OF_CIpo>A','PROCESS_OF_CIpro>A']
merge_edge_types(to_merge, 'LOCATION_OF_AloCI', swap=True)       

In [38]:
combo_num = 17

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Devices 

LOCATION_OF_AloDV         33917
NEG_LOCATION_OF_AnloDV      433
USES_AuDV                   126
PART_OF_Apo>DV               97
PART_OF_DVpo>A               71
TREATS_DVtA                  64
compared_with_DVcpwA         19
AFFECTS_DVafA                 8
CAUSES_DVcA                   7
DISRUPTS_DVdsA                4
AUGMENTS_DVagA                3
PRODUCES_Apd>DV               2
NEG_TREATS_DVntA              2
higher_than_DVhtA             2
NEG_PART_OF_AnpoDV            2
METHOD_OF_DVmoA               2
PREVENTS_DVpvA                1
ADMINISTERED_TO_DVatA         1
NEG_AUGMENTS_DVnagA           1
lower_than_DVltA              1
ASSOCIATED_WITH_AawDV         1
NEG_USES_AnuDV                1
COEXISTS_WITH_AcwDV           1
Name: TYPE, dtype: int64


Location of is the only useable edge, no merges needed

In [39]:
combo_num = 18

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Disorders 

LOCATION_OF_AloDO             500137
PART_OF_DOpo>A                 57619
PART_OF_Apo>DO                 20042
AFFECTS_DOafA                  15019
NEG_LOCATION_OF_AnloDO         10574
ASSOCIATED_WITH_AawDO           9586
CAUSES_AcDO                     5788
PRODUCES_DOpd>A                 3900
DISRUPTS_DOdsA                  1812
NEG_PART_OF_DOnpoA               738
COEXISTS_WITH_AcwDO              607
NEG_AFFECTS_AnafDO               574
PREDISPOSES_DOpsA                373
NEG_ASSOCIATED_WITH_AnawDO       279
NEG_CAUSES_AncDO                 200
TREATS_AtDO                      185
PROCESS_OF_DOpro>A               183
AUGMENTS_AagDO                   111
NEG_PRODUCES_DOnpdA              103
MANIFESTATION_OF_DOmfo>A          54
NEG_DISRUPTS_DOndsA               48
PRODUCES_Apd>DO                   43
DIAGNOSES_AdgDO                   40
PREVENTS_ApvDO                    23
PRECEDES_Apc>DO                   15
ISA_DOi>A                         12
NEG_PREDISPOSE

Several large count edges here, all will be merged to 'location_of'

In [40]:
to_merge = ['LOCATION_OF_AloDO', 'PART_OF_Apo>DO','ASSOCIATED_WITH_AawDO', 'PRODUCES_Apd>DO']
merge_edge_types(to_merge, 'LOCATION_OF_AloDO')

to_merge = ['PART_OF_DOpo>A',  'AFFECTS_DOafA', 'PROCESS_OF_DOpro>A']
merge_edge_types(to_merge, 'LOCATION_OF_AloDO', swap=True)

In [41]:
combo_num = 19

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Genes & Molecular Sequences 

LOCATION_OF_AloG             419681
PART_OF_Gpo>A                234502
PRODUCES_Apd>G                72412
AFFECTS_GafA                  68662
AUGMENTS_GagA                 68194
DISRUPTS_GdsA                 28793
NEG_LOCATION_OF_AnloG         10738
NEG_PART_OF_GnpoA              4658
NEG_PRODUCES_AnpdG             4569
NEG_AFFECTS_GnafA              4437
PRODUCES_Gpd>A                 2772
ADMINISTERED_TO_GatA           2461
NEG_AUGMENTS_GnagA             1857
COEXISTS_WITH_GcwA             1798
NEG_DISRUPTS_GndsA              747
INTERACTS_WITH_AiwG             519
INHIBITS_GinA                   127
STIMULATES_GstA                 118
PART_OF_Apo>G                   110
CAUSES_GcA                      103
ISA_Ai>G                         72
ASSOCIATED_WITH_GawA             62
compared_with_GcpwA              28
TREATS_GtA                       27
NEG_INTERACTS_WITH_GniwA         25
USES_AuG                         22
NEG_COEXISTS_WITH_Ancw

Here we will try for 3 associations:

1. Associated With (or localizes to)
2. Positive assocation (upregulated in)
3. Negative Association (downregulated in)

In [42]:
to_merge = ['LOCATION_OF_AloG', 'PRODUCES_Apd>G']
merge_edge_types(to_merge, 'LOCATION_OF_AloG')
to_merge = ['ASSOCIATED_WITH_GawA', 'COEXISTS_WITH_GcwA', 'PART_OF_Gpo>A']
merge_edge_types(to_merge, 'LOCATION_OF_AloG', swap=True)

 
to_merge = ['AFFECTS_GafA', 'AUGMENTS_GagA', 'DISRUPTS_GdsA', 'INHIBITS_GinA', 'STIMULATES_GstA']
merge_edge_types(to_merge, 'AFFECTS_GafA')
change_edge_type('INTERACTS_WITH_AiwG', 'AFFECTS_GafA', swap=True)


to_merge = ['NEG_LOCATION_OF_AnloG', 'NEG_PRODUCES_AnpdG', 'NEG_COEXISTS_WITH_AncwG']
merge_edge_types(to_merge, 'NEG_LOCATION_OF_AnloG')
to_merge = ['NEG_ASSOCIATED_WITH_GnawA','NEG_PART_OF_GnpoA']
merge_edge_types(to_merge, 'NEG_LOCATION_OF_AnloG', swap=True)

In [43]:
combo_num = 20

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Geographic Areas 

LOCATION_OF_GAloA      26
AFFECTS_GAafA           2
PART_OF_Apo>GA          1
PART_OF_GApo>A          1
NEG_AUGMENTS_GAnagA     1
Name: TYPE, dtype: int64


Too Small

In [44]:
combo_num = 21

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Living Beings 

PART_OF_Apo>LB                164385
LOCATION_OF_AloLB              40804
NEG_PART_OF_AnpoLB              1622
NEG_LOCATION_OF_AnloLB           891
PROCESS_OF_Apro>LB               678
PART_OF_LBpo>A                   400
AFFECTS_LBafA                    126
TREATS_AtLB                       99
ASSOCIATED_WITH_AawLB             64
ADMINISTERED_TO_LBatA             49
USES_LBuA                         34
PRODUCES_Apd>LB                   32
PRODUCES_LBpd>A                   27
AUGMENTS_LBagA                    27
NEG_PROCESS_OF_AnproLB            23
OCCURS_IN_AoiLB                   18
CAUSES_LBcA                       17
DIAGNOSES_LBdgA                   10
DISRUPTS_LBdsA                     8
INTERACTS_WITH_AiwLB               8
NEG_AFFECTS_AnafLB                 7
COEXISTS_WITH_LBcwA                3
ISA_Ai>LB                          2
NEG_PRODUCES_AnpdLB                2
NEG_TREATS_AntLB                   2
NEG_COEXISTS_WITH_LBncwA           1
NEG_AUGMEN

After spotchecking several instantances of each of part_of and location_of;

`Part_of` shows a partiuclar anatomy belongs to a particular living being  
`Location_of` shows where a particular living being (usually a microorganism) occurs

We will change Part_of to occurs_in to remove the directionality of the edge, which is unimportant for this study.

In [45]:
to_merge = ['OCCURS_IN_AoiLB' 'PART_OF_Apo>LB']
merge_edge_types(to_merge, 'OCCURS_IN_AoiLB')

In [46]:
combo_num = 22

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Objects 

LOCATION_OF_AloOB         3031
PART_OF_OBpo>A             332
AFFECTS_OBafA              242
AUGMENTS_OBagA             107
PART_OF_Apo>OB              60
DISRUPTS_OBdsA              58
NEG_LOCATION_OF_AnloOB      39
PRODUCES_Apd>OB             25
NEG_AFFECTS_OBnafA          25
USES_AuOB                   14
ISA_Ai>OB                    4
ADMINISTERED_TO_OBatA        3
TREATS_OBtA                  3
compared_with_OBcpwA         3
NEG_PART_OF_OBnpoA           3
INTERACTS_WITH_OBiwA         2
PRODUCES_OBpd>A              2
NEG_PRODUCES_AnpdOB          2
NEG_CAUSES_OBncA             1
NEG_AUGMENTS_OBnagA          1
CAUSES_OBcA                  1
higher_than_OBhtA            1
INHIBITS_OBinA               1
ASSOCIATED_WITH_OBawA        1
Name: TYPE, dtype: int64


Too small for any need

In [47]:
combo_num = 23

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Occupations 

LOCATION_OF_OCloA     59
METHOD_OF_AmoOC        7
PART_OF_Apo>OC         3
AFFECTS_OCafA          1
PROCESS_OF_OCpro>A     1
TREATS_OCtA            1
CAUSES_AcOC            1
Name: TYPE, dtype: int64


In [48]:
combo_num = 24

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Organizations 

LOCATION_OF_AloOR    8
PART_OF_Apo>OR       6
PART_OF_ORpo>A       6
AUGMENTS_ORagA       4
NEG_USES_AnuOR       1
Name: TYPE, dtype: int64


In [49]:
combo_num = 25

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Phenomena 

LOCATION_OF_AloPH             6824
AFFECTS_AafPH                  400
PROCESS_OF_Apro>PH             158
ASSOCIATED_WITH_AawPH          136
NEG_LOCATION_OF_AnloPH         121
PART_OF_PHpo>A                 103
PART_OF_Apo>PH                  59
AUGMENTS_PHagA                  53
PRODUCES_PHpd>A                 42
PROCESS_OF_PHpro>A              34
NEG_AFFECTS_PHnafA              33
DISRUPTS_PHdsA                  28
PRODUCES_Apd>PH                 15
CAUSES_PHcA                      9
NEG_PROCESS_OF_AnproPH           5
COEXISTS_WITH_AcwPH              4
NEG_ASSOCIATED_WITH_AnawPH       3
DIAGNOSES_PHdgA                  1
NEG_DISRUPTS_PHndsA              1
ISA_PHi>A                        1
NEG_PRODUCES_PHnpdA              1
TREATS_AtPH                      1
MANIFESTATION_OF_PHmfo>A         1
NEG_COEXISTS_WITH_AncwPH         1
PREDISPOSES_ApsPH                1
Name: TYPE, dtype: int64


In [50]:
combo_num = 26

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Physiology 

AFFECTS_AafPS                 18336
PROCESS_OF_PSpro>A             8893
LOCATION_OF_AloPS              3490
PRODUCES_PSpd>A                1301
NEG_AFFECTS_AnafPS             1265
PART_OF_PSpo>A                  459
PART_OF_Apo>PS                  452
ASSOCIATED_WITH_AawPS           191
NEG_PROCESS_OF_PSnproA          167
COEXISTS_WITH_AcwPS             145
CAUSES_PScA                      75
AUGMENTS_PSagA                   74
NEG_LOCATION_OF_AnloPS           70
PROCESS_OF_Apro>PS               67
DISRUPTS_PSdsA                   50
NEG_PRODUCES_PSnpdA              43
PRODUCES_Apd>PS                  38
PRECEDES_PSpc>A                  22
MANIFESTATION_OF_Amfo>PS         12
PRECEDES_Apc>PS                   9
TREATS_AtPS                       9
NEG_COEXISTS_WITH_AncwPS          8
NEG_ASSOCIATED_WITH_PSnawA        6
NEG_PART_OF_PSnpoA                6
MANIFESTATION_OF_PSmfo>A          4
NEG_CAUSES_AncPS                  2
NEG_DISRUPTS_PSndsA               2
ISA

In [51]:
# After looking closely at a few examples, Affects and Location_of are similar and should be combined
merge_edge_types(['AFFECTS_AafPS', 'ASSOCIATED_WITH_AawPS', 'LOCATION_OF_AloPS'],  'LOCATION_OF_AloPS')

merge_edge_types(['PROCESS_OF_PSpro>A', 'PART_OF_PSpo>A'],  'LOCATION_OF_AloPS', swap=True)

In [52]:
combo_num = 27

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Anatomy --- Procedures 

LOCATION_OF_AloPR             71555
NEG_LOCATION_OF_AnloPR         1472
TREATS_PRtA                     329
DIAGNOSES_PRdgA                 220
DISRUPTS_PRdsA                  181
AFFECTS_PRafA                   179
METHOD_OF_PRmoA                 161
PART_OF_Apo>PR                  157
USES_PRuA                       104
PART_OF_PRpo>A                  101
PRECEDES_PRpc>A                  56
ASSOCIATED_WITH_AawPR            44
STIMULATES_PRstA                 41
PRODUCES_PRpd>A                  33
CAUSES_PRcA                      30
NEG_AFFECTS_AnafPR               24
AUGMENTS_PRagA                   24
compared_with_PRcpwA             20
PREVENTS_PRpvA                   16
NEG_DISRUPTS_PRndsA              15
PRODUCES_Apd>PR                  14
PROCESS_OF_PRpro>A               14
COEXISTS_WITH_AcwPR              14
PRECEDES_Apc>PR                   8
NEG_TREATS_PRntA                  7
NEG_DIAGNOSES_PRndgA              6
higher_than_PRhtA                 5
PRE

In [53]:
combo_num = 28

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Chemicals & Drugs 

INTERACTS_WITH_CDiwCD         969797
COEXISTS_WITH_CDcwCD          540909
STIMULATES_CDstCD             512348
INHIBITS_CDinCD               438768
compared_with_CDcpwCD         217133
ISA_CDi>CD                    126972
PART_OF_CDpo>CD               119819
NEG_INTERACTS_WITH_CDniwCD     76813
PRODUCES_CDpd>CD               71898
higher_than_CDhtCD             70076
CONVERTS_TO_CDct>CD            36094
NEG_STIMULATES_CDnstCD         22429
NEG_INHIBITS_CDninCD           20433
USES_CDuCD                     18016
NEG_COEXISTS_WITH_CDncwCD      14950
same_as_CDsaCD                 14815
lower_than_CDltCD              14592
different_from_CDdfCD           3867
NEG_PART_OF_CDnpoCD             3208
NEG_PRODUCES_CDnpdCD            2547
NEG_higher_than_CDnhtCD         1913
LOCATION_OF_CDloCD              1903
different_than_CDdtCD            978
AFFECTS_CDafCD                   426
NEG_CONVERTS_TO_CDnctCD          411
NEG_lower_than_CDnltCD           

In [54]:
# These edges show compounds that are associated with each other...
to_merge = ['INTERACTS_WITH_CDiwCD', 'COEXISTS_WITH_CDcwCD', 'CONVERTS_TO_CDct>CD', 'PRODUCES_CDpd>CD', 'USES_CDuCD']
merge_edge_types(to_merge, 'ASSOCIATED_WITH_CDawCD')

# These should be directed edges
change_edge_type('STIMULATES_CDstCD', 'STIMULATES_CDst>CD')
change_edge_type('INHIBITS_CDinCD', 'INHIBITS_CDinCD')
 
# Merge to is-a
to_merge = ['ISA_CDi>CD', 'PART_OF_CDpo>CD']
merge_edge_types(to_merge, 'ISA_CDi>CD')
 
# Negation edges aren't too useful.  Its the same as having a 0 in the postivie edge
to_drop = ['NEG_INTERACTS_WITH_CDniwCD', 'NEG_STIMULATES_CDnstCD', 'NEG_INHIBITS_CDninCD', 'NEG_COEXISTS_WITH_CDncwCD']
drop_edges_from_list(to_drop)
    
# These comparison edges are also not useful, they will be dropped.
to_drop = ['compared_with_CDcpwCD', 'higher_than_CDhtCD','same_as_CDsaCD','lower_than_CDltCD']
drop_edges_from_list(to_drop)

In [55]:
combo_num = 29

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Concepts & Ideas 

AFFECTS_CDafCI                 5566
AUGMENTS_CDagCI                4700
CAUSES_CDcCI                   4255
USES_CIuCD                     3692
TREATS_CDtCI                   3011
DISRUPTS_CDdsCI                2459
INHIBITS_CDinCI                1631
ASSOCIATED_WITH_CDawCI         1336
PART_OF_CDpo>CI                1321
ADMINISTERED_TO_CDatCI         1139
LOCATION_OF_CIloCD             1089
INTERACTS_WITH_CIiwCD           942
NEG_AFFECTS_CDnafCI             849
PRODUCES_CIpd>CD                713
COEXISTS_WITH_CDcwCI            677
PREVENTS_CDpvCI                 567
PREDISPOSES_CDpsCI              424
STIMULATES_CIstCD               392
compared_with_CDcpwCI           348
NEG_CAUSES_CDncCI               180
NEG_AUGMENTS_CDnagCI            164
NEG_TREATS_CDntCI               128
NEG_DISRUPTS_CDndsCI            114
higher_than_CDhtCI               96
NEG_ASSOCIATED_WITH_CDnawCI      86
NEG_USES_CInuCD                  74
NEG_PREVENTS_CDnpvCI   

In [56]:
# May result in enough instanfces for edge to pass filtering if we combine all 
# modulation relationships
to_merge = ['AFFECTS_CDafCI','AUGMENTS_CDagCI', 'CAUSES_CDcCI',
       'TREATS_CDtCI', 'DISRUPTS_CDdsCI', 'INHIBITS_CDinCI',
       'ASSOCIATED_WITH_CDawCI', 'PREVENTS_CDpvCI','PREDISPOSES_CDpsCI']
merge_edge_types(to_merge, 'AFFECTS_CDafCI')


In [57]:
combo_num = 30

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Devices 

USES_DVuCD                    18592
compared_with_CDcpwDV          8379
higher_than_DVhtCD             1786
same_as_CDsaDV                  364
lower_than_DVltCD               334
NEG_USES_DVnuCD                 157
LOCATION_OF_DVloCD              125
INTERACTS_WITH_DViwCD           120
ISA_DVi>CD                       90
COEXISTS_WITH_CDcwDV             67
different_from_DVdfCD            60
PART_OF_CDpo>DV                  54
NEG_higher_than_CDnhtDV          50
ISA_CDi>DV                       49
AFFECTS_CDafDV                   44
STIMULATES_DVstCD                21
AUGMENTS_CDagDV                  21
PART_OF_DVpo>CD                  15
INHIBITS_DVinCD                  15
different_than_CDdtDV            14
ASSOCIATED_WITH_CDawDV           13
PRODUCES_DVpd>CD                 12
TREATS_DVtCD                     11
DIAGNOSES_DVdgCD                  9
NEG_lower_than_DVnltCD            5
NEG_INTERACTS_WITH_CDniwDV        5
DISRUPTS_CDdsDV                 

Nothing to merge here.  Uses is the only decent edge, but may not have enough counts to pass filtering.

In [58]:
combo_num = 31

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Disorders 

TREATS_CDtDO                   503816
ASSOCIATED_WITH_CDawDO         407482
CAUSES_CDcDO                   287076
AFFECTS_CDafDO                 227096
AUGMENTS_CDagDO                 99814
PREVENTS_CDpvDO                 98261
DISRUPTS_CDdsDO                 93382
PREDISPOSES_CDpsDO              88371
PRODUCES_DOpd>CD                40772
PART_OF_CDpo>DO                 33136
NEG_AFFECTS_CDnafDO             20193
INHIBITS_CDinDO                 17343
NEG_CAUSES_CDncDO               12667
NEG_TREATS_CDntDO               12500
NEG_ASSOCIATED_WITH_CDnawDO     11124
NEG_AUGMENTS_CDnagDO             4786
NEG_PREVENTS_CDnpvDO             4415
NEG_PREDISPOSES_CDnpsDO          4003
COMPLICATES_CDcpDO               3524
NEG_DISRUPTS_CDndsDO             3187
LOCATION_OF_DOloCD               2965
USES_DOuCD                       1900
NEG_PRODUCES_DOnpdCD             1515
NEG_INHIBITS_CDninDO              942
NEG_PART_OF_CDnpoDO               449
ADMINISTERED_TO_

In [59]:
# Negatvie effect
to_merge = ['TREATS_CDtDO', 'PREVENTS_CDpvDO','DISRUPTS_CDdsDO', 'INHIBITS_CDinDO']
merge_edge_types(to_merge, to_type='TREATS_CDtDO')

# General Association
to_merge = ['ASSOCIATED_WITH_CDawDO', 'AFFECTS_CDafDO', 'PART_OF_CDpo>DO']
merge_edge_types(to_merge, to_type='ASSOCIATED_WITH_CDawDO')
change_edge_type('PRODUCES_DOpd>CD', 'ASSOCIATED_WITH_CDawDO', swap=True)

# Positive effect
to_merge = ['CAUSES_CDcDO', 'AUGMENTS_CDagDO','PREDISPOSES_CDpsDO', 'COMPLICATES_CDcpDO']
merge_edge_types(to_merge, 'AUGMENTS_CDagDO')
        
# Drop the large negation edges
to_drop = ['NEG_AFFECTS_CDnafDO', 'NEG_CAUSES_CDncDO', 'NEG_TREATS_CDntDO', 'NEG_ASSOCIATED_WITH_CDnawDO',
       'NEG_AUGMENTS_CDnagDO', 'NEG_PREVENTS_CDnpvDO','NEG_PREDISPOSES_CDnpsDO', 'NEG_DISRUPTS_CDndsDO']
drop_edges_from_list(to_drop) 

In [60]:
combo_num = 32

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Genes & Molecular Sequences 

INTERACTS_WITH_GiwCD         447759
STIMULATES_GstCD             333876
COEXISTS_WITH_CDcwG          268959
INHIBITS_CDinG               242761
PART_OF_CDpo>G               119075
compared_with_GcpwCD          69058
PRODUCES_Gpd>CD               55705
NEG_INTERACTS_WITH_CDniwG     36268
PRODUCES_CDpd>G               33083
PART_OF_Gpo>CD                23327
higher_than_CDhtG             18707
ISA_Gi>CD                     15155
NEG_STIMULATES_CDnstG         14621
NEG_INHIBITS_CDninG            8999
NEG_COEXISTS_WITH_GncwCD       8909
CONVERTS_TO_Gct>CD             8262
CONVERTS_TO_CDct>G             7027
USES_CDuG                      5654
ISA_CDi>G                      5475
same_as_GsaCD                  3596
lower_than_CDltG               3267
NEG_PRODUCES_GnpdCD            3110
NEG_PART_OF_CDnpoG             2773
different_from_CDdfG           1479
LOCATION_OF_GloCD               591
NEG_higher_than_GnhtCD          378
AFFECTS_CDaf

In [61]:
# Large amount, but all show a general relationship between Gene and Chemical.
to_merge = ['INTERACTS_WITH_GiwCD', 'PART_OF_Gpo>CD','ISA_Gi>CD', 'CONVERTS_TO_Gct>CD']
merge_edge_types(to_merge, 'INTERACTS_WITH_CDiwG', swap=True)
to_merge = ['COEXISTS_WITH_CDcwG', 'PART_OF_CDpo>G',  'USES_CDuG','ISA_CDi>G', 'CONVERTS_TO_CDct>G']
merge_edge_types(to_merge, 'INTERACTS_WITH_CDiwG')

# Merge Stimulates edges
to_merge = ['STIMULATES_GstCD', 'PRODUCES_Gpd>CD']
merge_edge_types(to_merge, 'STIMULATES_CDstG', swap=True)
to_merge = ['PRODUCES_CDpd>G']
merge_edge_types(to_merge, 'STIMULATES_CDstG')

# keep inhibits as-is

#drop useless edges
to_drop = ['higher_than_CDhtG', 'NEG_STIMULATES_CDnstG', 'compared_with_GcpwCD', 'NEG_INTERACTS_WITH_CDniwG']
drop_edges_from_list(to_drop)

In [62]:
combo_num = 33

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Geographic Areas 

COEXISTS_WITH_CDcwGA     15
INTERACTS_WITH_CDiwGA     1
LOCATION_OF_GAloCD        1
Name: TYPE, dtype: int64


In [63]:
combo_num = 34

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Living Beings 

LOCATION_OF_LBloCD             397815
PART_OF_CDpo>LB                198444
TREATS_CDtLB                   149167
ADMINISTERED_TO_CDatLB          95641
INTERACTS_WITH_CDiwLB           12287
PRODUCES_LBpd>CD                10797
NEG_LOCATION_OF_LBnloCD          7105
NEG_TREATS_CDntLB                5461
NEG_ADMINISTERED_TO_CDnatLB      4348
NEG_PART_OF_CDnpoLB              3034
AFFECTS_CDafLB                   2109
USES_LBuCD                       1282
PREVENTS_CDpvLB                  1161
AUGMENTS_CDagLB                   452
COEXISTS_WITH_CDcwLB              344
NEG_INTERACTS_WITH_LBniwCD        332
NEG_PRODUCES_LBnpdCD              331
DISRUPTS_CDdsLB                   289
PROCESS_OF_CDpro>LB               225
NEG_AFFECTS_CDnafLB               113
STIMULATES_CDstLB                 112
INHIBITS_LBinCD                    71
compared_with_CDcpwLB              64
ASSOCIATED_WITH_CDawLB             40
NEG_PREVENTS_CDnpvLB               40
CAUSES_CDcLB

In [64]:
# split into two types... naturally occuring, and added to.
to_merge = ['LOCATION_OF_LBloCD', 'PRODUCES_LBpd>CD']
merge_edge_types(to_merge, 'PRODUCES_LBpdCD')
change_edge_type('PART_OF_CDpo>LB', 'PRODUCES_LBpdCD', swap=True)
 
    
# 'TREATS_CDtLB' is potentially pathogens tereated by a combpound

# 'ADMINISTERED_TO_CDatLB' is more like the beign that a drug is given to...?

In [65]:
combo_num = 35

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Objects 

COEXISTS_WITH_OBcwCD          2997
INTERACTS_WITH_OBiwCD         2691
compared_with_CDcpwOB         2261
USES_OBuCD                    1850
ISA_CDi>OB                    1623
LOCATION_OF_OBloCD            1244
STIMULATES_CDstOB              977
INHIBITS_OBinCD                821
higher_than_CDhtOB             522
ISA_OBi>CD                     292
NEG_INTERACTS_WITH_OBniwCD     225
same_as_OBsaCD                  92
CONVERTS_TO_OBct>CD             87
lower_than_OBltCD               85
PART_OF_CDpo>OB                 81
PRODUCES_CDpd>OB                57
NEG_COEXISTS_WITH_CDncwOB       49
PART_OF_OBpo>CD                 45
NEG_STIMULATES_OBnstCD          30
CONVERTS_TO_CDct>OB             27
PRODUCES_OBpd>CD                27
NEG_INHIBITS_OBninCD            24
NEG_USES_CDnuOB                 22
different_from_CDdfOB           16
NEG_higher_than_OBnhtCD         16
NEG_LOCATION_OF_OBnloCD         15
AFFECTS_CDafOB                  11
different_than_OBdtCD  

not enough counts to matter

In [66]:
combo_num = 36

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Occupations 

AFFECTS_CDafOC            615
USES_OCuCD                229
METHOD_OF_CDmoOC           86
AUGMENTS_CDagOC            60
DISRUPTS_CDdsOC            58
ASSOCIATED_WITH_CDawOC     36
NEG_AFFECTS_CDnafOC        24
CAUSES_CDcOC               23
COEXISTS_WITH_CDcwOC       13
TREATS_CDtOC               11
PRODUCES_OCpd>CD           10
INTERACTS_WITH_OCiwCD       9
compared_with_OCcpwCD       7
LOCATION_OF_OCloCD          7
ADMINISTERED_TO_CDatOC      5
PART_OF_CDpo>OC             2
NEG_TREATS_CDntOC           1
PREVENTS_CDpvOC             1
NEG_METHOD_OF_CDnmoOC       1
NEG_USES_OCnuCD             1
higher_than_OChtCD          1
STIMULATES_CDstOC           1
Name: TYPE, dtype: int64


In [67]:
combo_num = 37

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Organizations 

LOCATION_OF_ORloCD        42
PART_OF_CDpo>OR           28
USES_CDuOR                10
TREATS_CDtOR               5
AFFECTS_CDafOR             5
COEXISTS_WITH_CDcwOR       3
INTERACTS_WITH_CDiwOR      3
ADMINISTERED_TO_CDatOR     2
AUGMENTS_CDagOR            2
PRODUCES_ORpd>CD           2
STIMULATES_CDstOR          1
compared_with_ORcpwCD      1
INHIBITS_ORinCD            1
Name: TYPE, dtype: int64


In [68]:
combo_num = 38

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Phenomena 

AFFECTS_CDafPH                 46044
AUGMENTS_CDagPH                24534
DISRUPTS_CDdsPH                15913
CAUSES_CDcPH                    5143
NEG_AFFECTS_CDnafPH             4182
PRODUCES_PHpd>CD                1524
NEG_AUGMENTS_CDnagPH             986
TREATS_CDtPH                     927
LOCATION_OF_CDloPH               896
INTERACTS_WITH_PHiwCD            873
NEG_DISRUPTS_CDndsPH             702
INHIBITS_CDinPH                  650
PART_OF_CDpo>PH                  495
USES_PHuCD                       409
STIMULATES_PHstCD                364
COEXISTS_WITH_PHcwCD             309
NEG_CAUSES_CDncPH                173
ASSOCIATED_WITH_PHawCD           145
compared_with_PHcpwCD             74
NEG_INTERACTS_WITH_PHniwCD        65
NEG_PRODUCES_PHnpdCD              45
ADMINISTERED_TO_CDatPH            34
NEG_TREATS_CDntPH                 33
PREDISPOSES_CDpsPH                20
NEG_LOCATION_OF_PHnloCD           19
NEG_INHIBITS_CDninPH              18
PROD

In [69]:
# We could have a +, -, and assc relations here, but one edge of just AFFECTS may have more power

to_merge = ['AFFECTS_CDafPH', 'AUGMENTS_CDagPH', 'DISRUPTS_CDdsPH', 'CAUSES_CDcPH']
merge_edge_types(to_merge, 'AFFECTS_CDafPH')

In [70]:
combo_num = 39

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Physiology 

AFFECTS_CDafPS                 367287
AUGMENTS_CDagPS                185771
DISRUPTS_CDdsPS                170967
CAUSES_CDcPS                    64115
NEG_AFFECTS_CDnafPS             43357
PRODUCES_PSpd>CD                28381
INHIBITS_CDinPS                 17703
NEG_DISRUPTS_CDndsPS             9752
NEG_AUGMENTS_CDnagPS             9143
ASSOCIATED_WITH_PSawCD           6168
NEG_CAUSES_CDncPS                3363
TREATS_CDtPS                     2888
LOCATION_OF_PSloCD               1543
NEG_INHIBITS_CDninPS             1070
STIMULATES_CDstPS                 941
USES_PSuCD                        934
NEG_PRODUCES_PSnpdCD              763
INTERACTS_WITH_PSiwCD             707
PREVENTS_CDpvPS                   493
COEXISTS_WITH_PScwCD              493
PART_OF_CDpo>PS                   444
COMPLICATES_CDcpPS                385
NEG_ASSOCIATED_WITH_PSnawCD       283
ADMINISTERED_TO_CDatPS            224
PREDISPOSES_CDpsPS                217
NEG_TREATS_CDnt

In [71]:
# Large Enough Numbers here that we can go with one of each.

# Just one neutral relationship, 'AFFECTS_CDafPS', 

# Merge the positives
to_merge = ['AUGMENTS_CDagPS', 'CAUSES_CDcPS', 'STIMULATES_CDstPS']
merge_edge_types(to_merge, 'AUGMENTS_CDagPS')

# Merge the negatives
to_merge = ['DISRUPTS_CDdsPS', 'INHIBITS_CDinPS', 'TREATS_CDtPS']
merge_edge_types(to_merge, 'DISRUPTS_CDdsPS')

# These are silgithly different, chemicals that come as a result of a physiology
to_merge = ['ASSOCIATED_WITH_PSawCD', 'PRODUCES_PSpd>CD']
merge_edge_types(to_merge, 'ASSOCIATED_WITH_PSawCD')

# Drop unhelpful edges
to_drop = ['NEG_AFFECTS_CDnafPS', 'NEG_DISRUPTS_CDndsPS', 'NEG_AUGMENTS_CDnagPS',
 'NEG_CAUSES_CDncPS','LOCATION_OF_PSloCD', 'NEG_INHIBITS_CDninPS']
drop_edges_from_list(to_drop)

In [72]:
combo_num = 40

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Chemicals & Drugs --- Procedures 

USES_PRuCD                     284820
compared_with_CDcpwPR           40733
ISA_CDi>PR                      15554
higher_than_CDhtPR              10603
METHOD_OF_CDmoPR                 8255
NEG_USES_PRnuCD                  5122
PRECEDES_CDpc>PR                 3928
same_as_CDsaPR                   2742
PRECEDES_PRpc>CD                 2554
AFFECTS_CDafPR                   2001
lower_than_CDltPR                1884
COEXISTS_WITH_PRcwCD             1828
AUGMENTS_CDagPR                  1466
ISA_PRi>CD                       1244
NEG_higher_than_CDnhtPR           887
INHIBITS_PRinCD                   875
TREATS_CDtPR                      840
DISRUPTS_CDdsPR                   747
PRODUCES_PRpd>CD                  621
CAUSES_CDcPR                      617
LOCATION_OF_PRloCD                556
NEG_AFFECTS_CDnafPR               494
INTERACTS_WITH_CDiwPR             479
different_from_PRdfCD             278
PART_OF_CDpo>PR                   256
STIMULATES_PRst

In [73]:
# Main edge: 'USES_PRuCD'
# These are close nough but need swap
to_merge = ['ISA_CDi>PR','METHOD_OF_CDmoPR']
merge_edge_types(to_merge, 'USES_PRuCD', swap=True)

# these are not useful
to_drop =  ['compared_with_CDcpwPR', 'higher_than_CDhtPR']
drop_edges_from_list(to_drop)

In [74]:
combo_num = 41

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Concepts & Ideas 

ISA_CIi>CI                     7770
USES_CIuCI                      175
LOCATION_OF_CIloCI              163
PROCESS_OF_CIpro>CI              73
METHOD_OF_CImoCI                 58
AFFECTS_CIafCI                   48
compared_with_CIcpwCI            38
TREATS_CItCI                     31
ADMINISTERED_TO_CIatCI           25
COEXISTS_WITH_CIcwCI             25
DIAGNOSES_CIdgCI                 24
OCCURS_IN_CIoiCI                 15
PRODUCES_CIpd>CI                 11
higher_than_CIhtCI               11
NEG_PROCESS_OF_CInproCI           7
CAUSES_CIcCI                      6
NEG_LOCATION_OF_CInloCI           5
same_as_CIsaCI                    5
PRECEDES_CIpc>CI                  5
lower_than_CIltCI                 4
NEG_USES_CInuCI                   4
NEG_AFFECTS_CInafCI               4
NEG_METHOD_OF_CInmoCI             3
INTERACTS_WITH_CIiwCI             3
ASSOCIATED_WITH_CIawCI            3
PART_OF_CIpo>CI                   3
PREDISPOSES_CIpsCI      

In [75]:
combo_num = 42

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Devices 

USES_CIuDV                 4291
TREATS_DVtCI                512
LOCATION_OF_CIloDV          110
NEG_USES_CInuDV              74
CAUSES_DVcCI                 59
compared_with_DVcpwCI        56
PRODUCES_CIpd>DV             50
ISA_CIi>DV                   47
PREVENTS_DVpvCI              19
ADMINISTERED_TO_DVatCI        9
higher_than_DVhtCI            9
ISA_DVi>CI                    9
NEG_TREATS_DVntCI             7
METHOD_OF_DVmoCI              4
NEG_CAUSES_DVncCI             3
same_as_DVsaCI                2
AFFECTS_DVafCI                1
PROCESS_OF_DVpro>CI           1
NEG_LOCATION_OF_CInloDV       1
PART_OF_CIpo>DV               1
NEG_PRODUCES_CInpdDV          1
PRECEDES_DVpc>CI              1
lower_than_CIltDV             1
DIAGNOSES_DVdgCI              1
Name: TYPE, dtype: int64


In [76]:
combo_num = 43

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Disorders 

PROCESS_OF_DOpro>CI              5781
DIAGNOSES_CIdgDO                 5391
COEXISTS_WITH_CIcwDO             5256
TREATS_CItDO                     3362
AFFECTS_CIafDO                   3347
LOCATION_OF_CIloDO               2408
ASSOCIATED_WITH_CIawDO           1525
CAUSES_DOcCI                     1438
PREDISPOSES_CIpsDO               1371
PART_OF_DOpo>CI                   352
PROCESS_OF_CIpro>DO               308
PREVENTS_CIpvDO                   291
NEG_AFFECTS_DOnafCI               211
NEG_COEXISTS_WITH_DOncwCI         205
MANIFESTATION_OF_DOmfo>CI         194
ISA_DOi>CI                        192
OCCURS_IN_CIoiDO                  170
NEG_DIAGNOSES_CIndgDO             156
NEG_PROCESS_OF_DOnproCI           142
MANIFESTATION_OF_CImfo>DO         115
INTERACTS_WITH_DOiwCI             114
ISA_CIi>DO                         98
COMPLICATES_CIcpDO                 82
PRECEDES_CIpc>DO                   82
NEG_LOCATION_OF_CInloDO            66
USES_DOuCI       

If all were merged to one egde type, could be useable, but the edges are different enoguh that merge doesn't seem applicable.

In [77]:
combo_num = 44

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Genes & Molecular Sequences 

AFFECTS_GafCI                 1688
AUGMENTS_GagCI                1444
CAUSES_GcCI                   1306
ASSOCIATED_WITH_GawCI          870
USES_CIuG                      820
DISRUPTS_GdsCI                 731
PART_OF_Gpo>CI                 557
LOCATION_OF_CIloG              535
INTERACTS_WITH_GiwCI           293
COEXISTS_WITH_CIcwG            275
PRODUCES_CIpd>G                223
NEG_AFFECTS_GnafCI             194
PREDISPOSES_GpsCI              186
TREATS_GtCI                    178
STIMULATES_CIstG               164
INHIBITS_GinCI                 142
compared_with_CIcpwG           111
ADMINISTERED_TO_GatCI           65
NEG_CAUSES_GncCI                41
NEG_ASSOCIATED_WITH_GnawCI      40
PRODUCES_Gpd>CI                 38
NEG_AUGMENTS_GnagCI             34
PREVENTS_GpvCI                  31
higher_than_CIhtG               30
PART_OF_CIpo>G                  22
NEG_INTERACTS_WITH_CIniwG       19
NEG_PRODUCES_CInpdG             15
NEG_

In [78]:
combo_num = 45

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Geographic Areas 

ISA_GAi>CI                     5
ISA_CIi>GA                     1
NEG_ASSOCIATED_WITH_GAnawCI    1
Name: TYPE, dtype: int64


In [79]:
combo_num = 46

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Living Beings 

PROCESS_OF_CIpro>LB            5203
USES_LBuCI                     5072
AFFECTS_CIafLB                 1137
ADMINISTERED_TO_CIatLB         1114
TREATS_CItLB                   1036
PART_OF_CIpo>LB                 483
INTERACTS_WITH_LBiwCI           439
LOCATION_OF_LBloCI              385
PRODUCES_LBpd>CI                313
NEG_PROCESS_OF_CInproLB         210
NEG_USES_LBnuCI                 159
CAUSES_LBcCI                    132
ISA_LBi>CI                       94
DIAGNOSES_CIdgLB                 87
OCCURS_IN_CIoiLB                 76
ISA_CIi>LB                       65
PROCESS_OF_LBpro>CI              55
NEG_AFFECTS_CInafLB              45
NEG_ADMINISTERED_TO_CInatLB      31
NEG_TREATS_CIntLB                23
NEG_INTERACTS_WITH_CIniwLB       15
COEXISTS_WITH_LBcwCI             13
NEG_PRODUCES_LBnpdCI             10
PRODUCES_CIpd>LB                  9
NEG_PART_OF_CInpoLB               7
METHOD_OF_CImoLB                  7
NEG_CAUSES_LBncCI          

In [80]:
combo_num = 47

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Objects 

USES_CIuOB                     465
ISA_CIi>OB                     137
ISA_OBi>CI                     102
TREATS_OBtCI                    55
CAUSES_OBcCI                    55
AFFECTS_OBafCI                  38
LOCATION_OF_OBloCI              29
NEG_USES_CInuOB                 12
AUGMENTS_OBagCI                 12
PART_OF_OBpo>CI                 10
compared_with_OBcpwCI            5
COEXISTS_WITH_CIcwOB             5
INHIBITS_OBinCI                  4
NEG_CAUSES_OBncCI                3
NEG_AFFECTS_OBnafCI              3
PRODUCES_CIpd>OB                 3
PREDISPOSES_OBpsCI               2
ASSOCIATED_WITH_OBawCI           2
DISRUPTS_OBdsCI                  2
higher_than_CIhtOB               1
NEG_TREATS_OBntCI                1
PREVENTS_OBpvCI                  1
METHOD_OF_OBmoCI                 1
NEG_ASSOCIATED_WITH_OBnawCI      1
PART_OF_CIpo>OB                  1
PRODUCES_OBpd>CI                 1
ADMINISTERED_TO_OBatCI           1
INTERACTS_WITH_OBiwCI   

In [81]:
combo_num = 48

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Occupations 

METHOD_OF_CImoOC             716
ISA_CIi>OC                    15
NEG_METHOD_OF_CInmoOC          7
COEXISTS_WITH_CIcwOC           5
PROCESS_OF_CIpro>OC            4
AFFECTS_CIafOC                 4
TREATS_OCtCI                   3
ISA_OCi>CI                     3
USES_CIuOC                     2
ADMINISTERED_TO_OCatCI         2
CAUSES_OCcCI                   1
NEG_COEXISTS_WITH_CIncwOC      1
PRECEDES_CIpc>OC               1
LOCATION_OF_OCloCI             1
Name: TYPE, dtype: int64


In [82]:
combo_num = 49

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Organizations 

LOCATION_OF_ORloCI         2207
AFFECTS_CIafOR              260
PRODUCES_ORpd>CI             99
USES_CIuOR                   56
NEG_LOCATION_OF_ORnloCI      30
NEG_AFFECTS_CInafOR           8
ISA_CIi>OR                    6
PROCESS_OF_CIpro>OR           4
ISA_ORi>CI                    2
NEG_PRODUCES_ORnpdCI          1
ADMINISTERED_TO_CIatOR        1
Name: TYPE, dtype: int64


In [83]:
combo_num = 50

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Phenomena 

AFFECTS_CIafPH                   477
PROCESS_OF_CIpro>PH              383
PROCESS_OF_PHpro>CI              306
LOCATION_OF_CIloPH                76
NEG_AFFECTS_CInafPH               53
COEXISTS_WITH_PHcwCI              24
NEG_PROCESS_OF_CInproPH           22
TREATS_CItPH                      21
ISA_CIi>PH                        20
DIAGNOSES_PHdgCI                  12
CAUSES_PHcCI                      12
DISRUPTS_CIdsPH                   12
AUGMENTS_CIagPH                   10
OCCURS_IN_PHoiCI                   9
ISA_PHi>CI                         6
MANIFESTATION_OF_PHmfo>CI          6
USES_CIuPH                         6
MANIFESTATION_OF_CImfo>PH          5
METHOD_OF_PHmoCI                   5
PREDISPOSES_CIpsPH                 4
NEG_DIAGNOSES_CIndgPH              2
PRODUCES_CIpd>PH                   2
PREVENTS_PHpvCI                    2
NEG_PREVENTS_PHnpvCI               1
NEG_ASSOCIATED_WITH_CInawPH        1
INHIBITS_CIinPH                    1
NEG_A

In [84]:
combo_num = 51

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Physiology 

AFFECTS_CIafPS                   2475
OCCURS_IN_PSoiCI                 1149
PROCESS_OF_PSpro>CI               889
ISA_CIi>PS                        624
COEXISTS_WITH_PScwCI              547
PROCESS_OF_CIpro>PS               513
NEG_AFFECTS_CInafPS               252
MANIFESTATION_OF_CImfo>PS         215
CAUSES_CIcPS                      179
ISA_PSi>CI                        144
DISRUPTS_CIdsPS                    82
AUGMENTS_PSagCI                    77
TREATS_CItPS                       75
USES_PSuCI                         51
ASSOCIATED_WITH_PSawCI             47
NEG_PROCESS_OF_PSnproCI            44
NEG_OCCURS_IN_PSnoiCI              44
PRECEDES_PSpc>CI                   37
PREDISPOSES_CIpsPS                 36
LOCATION_OF_CIloPS                 32
ADMINISTERED_TO_CIatPS             32
STIMULATES_CIstPS                  32
DIAGNOSES_CIdgPS                   29
PRECEDES_CIpc>PS                   29
NEG_COEXISTS_WITH_CIncwPS          25
PRODUCES_PSpd>CI

In [85]:
combo_num = 52

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Concepts & Ideas --- Procedures 

TREATS_PRtCI                   3089
METHOD_OF_CImoPR               2798
USES_CIuPR                     1792
ADMINISTERED_TO_PRatCI         1511
DIAGNOSES_PRdgCI               1186
PRECEDES_PRpc>CI                587
AFFECTS_PRafCI                  547
LOCATION_OF_CIloPR              432
compared_with_CIcpwPR           428
PRECEDES_CIpc>PR                342
COEXISTS_WITH_PRcwCI            247
PREVENTS_PRpvCI                 208
ISA_CIi>PR                      182
ISA_PRi>CI                      161
higher_than_PRhtCI              113
NEG_TREATS_PRntCI               105
NEG_METHOD_OF_CInmoPR            97
CAUSES_PRcCI                     71
NEG_DIAGNOSES_PRndgCI            57
NEG_AFFECTS_PRnafCI              46
same_as_CIsaPR                   45
NEG_ADMINISTERED_TO_PRnatCI      38
lower_than_PRltCI                35
PREDISPOSES_PRpsCI               33
NEG_USES_PRnuCI                  30
PROCESS_OF_PRpro>CI              21
STIMULATES_PRstCI             

In [86]:
combo_num = 53

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Devices 

USES_DVuDV                 24813
compared_with_DVcpwDV       5203
ISA_DVi>DV                  1469
higher_than_DVhtDV          1341
lower_than_DVltDV            298
same_as_DVsaDV               294
NEG_USES_DVnuDV              293
NEG_higher_than_DVnhtDV       62
different_from_DVdfDV         43
LOCATION_OF_DVloDV            22
NEG_lower_than_DVnltDV        11
different_than_DVdtDV          7
TREATS_DVtDV                   4
METHOD_OF_DVmoDV               1
NEG_TREATS_DVntDV              1
PRODUCES_DVpd>DV               1
Name: TYPE, dtype: int64


In [87]:
to_merge = ['USES_DVuDV', 'ISA_DVi>DV']

merge_edge_types(to_merge, 'USES_DVuDV')

In [88]:
combo_num = 54

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Disorders 

TREATS_DVtDO                 58436
CAUSES_DVcDO                 12696
PREVENTS_DVpvDO               6607
NEG_TREATS_DVntDO              922
NEG_CAUSES_DVncDO              374
USES_DOuDV                     306
LOCATION_OF_DVloDO             268
NEG_PREVENTS_DVnpvDO           230
AFFECTS_DVafDO                 122
DIAGNOSES_DVdgDO                86
PREDISPOSES_DVpsDO              30
COEXISTS_WITH_DVcwDO            19
PART_OF_DOpo>DV                 19
PART_OF_DVpo>DO                 17
compared_with_DVcpwDO           15
ASSOCIATED_WITH_DVawDO          12
METHOD_OF_DVmoDO                 9
NEG_AFFECTS_DVnafDO              8
PROCESS_OF_DOpro>DV              7
PROCESS_OF_DVpro>DO              7
PRECEDES_DVpc>DO                 7
AUGMENTS_DVagDO                  6
NEG_LOCATION_OF_DVnloDO          5
NEG_PREDISPOSES_DVnpsDO          3
DISRUPTS_DVdsDO                  3
NEG_USES_DOnuDV                  3
PRODUCES_DOpd>DV                 2
INHIBITS_DVinDO                

In [89]:
to_merge = ['TREATS_DVtDO', 'PREVENTS_DVpvDO']
merge_edge_types(to_merge, 'TREATS_DVtDO')


# keep as is: 'CAUSES_DVcDO'


In [90]:
combo_num = 55

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Genes & Molecular Sequences 

compared_with_GcpwDV         2185
higher_than_GhtDV             541
USES_GuDV                     468
same_as_GsaDV                  73
lower_than_DVltG               69
PART_OF_Gpo>DV                 35
LOCATION_OF_DVloG              33
INTERACTS_WITH_DViwG           27
NEG_higher_than_GnhtDV         13
different_from_DVdfG           11
INHIBITS_DVinG                 10
COEXISTS_WITH_GcwDV             7
STIMULATES_DVstG                6
AFFECTS_GafDV                   6
different_than_GdtDV            5
PRODUCES_DVpd>G                 5
PART_OF_DVpo>G                  5
NEG_lower_than_GnltDV           4
AUGMENTS_GagDV                  4
ASSOCIATED_WITH_DVawG           2
NEG_INTERACTS_WITH_GniwDV       1
NEG_COEXISTS_WITH_GncwDV        1
TREATS_DVtG                     1
DISRUPTS_GdsDV                  1
NEG_USES_DVnuG                  1
PRECEDES_DVpc>G                 1
Name: TYPE, dtype: int64


In [91]:
combo_num = 56

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Geographic Areas 

LOCATION_OF_GAloDV    1
Name: TYPE, dtype: int64


In [92]:
combo_num = 57

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Living Beings 

TREATS_DVtLB                   20326
USES_LBuDV                     15396
PRODUCES_LBpd>DV                 740
DIAGNOSES_DVdgLB                 666
NEG_USES_LBnuDV                  584
NEG_TREATS_DVntLB                469
ADMINISTERED_TO_DVatLB           316
PART_OF_DVpo>LB                   54
AFFECTS_DVafLB                    25
PROCESS_OF_DVpro>LB               24
LOCATION_OF_DVloLB                14
NEG_ADMINISTERED_TO_DVnatLB       12
NEG_DIAGNOSES_DVndgLB             11
NEG_PRODUCES_LBnpdDV               8
compared_with_DVcpwLB              2
lower_than_DVltLB                  1
higher_than_LBhtDV                 1
INTERACTS_WITH_DViwLB              1
CAUSES_DVcLB                       1
NEG_INTERACTS_WITH_LBniwDV         1
PREVENTS_DVpvLB                    1
Name: TYPE, dtype: int64


These are fine as-is, nothuing to combine

In [93]:
combo_num = 58

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Objects 

USES_DVuOB                 14775
compared_with_DVcpwOB        537
NEG_USES_DVnuOB              232
higher_than_OBhtDV           114
ISA_DVi>OB                   102
ISA_OBi>DV                    49
lower_than_DVltOB             23
LOCATION_OF_OBloDV            16
same_as_OBsaDV                14
NEG_higher_than_DVnhtOB        4
TREATS_OBtDV                   2
different_from_DVdfOB          2
NEG_LOCATION_OF_OBnloDV        1
Name: TYPE, dtype: int64


In [94]:
combo_num = 59

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Occupations 

USES_OCuDV              46
METHOD_OF_DVmoOC         8
TREATS_DVtOC             7
COEXISTS_WITH_DVcwOC     2
Name: TYPE, dtype: int64


In [95]:
combo_num = 60

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Organizations 

USES_DVuOR            1608
NEG_USES_DVnuOR         12
LOCATION_OF_ORloDV      10
ISA_DVi>OR               2
TREATS_DVtOR             1
Name: TYPE, dtype: int64


In [96]:
combo_num = 61

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Phenomena 

USES_PHuDV                 123
TREATS_DVtPH               104
PROCESS_OF_DVpro>PH         14
AFFECTS_DVafPH              11
PROCESS_OF_PHpro>DV          8
LOCATION_OF_DVloPH           7
PREVENTS_DVpvPH              3
AUGMENTS_DVagPH              2
NEG_TREATS_DVntPH            2
NEG_USES_PHnuDV              2
compared_with_DVcpwPH        1
CAUSES_DVcPH                 1
higher_than_DVhtPH           1
NEG_LOCATION_OF_DVnloPH      1
Name: TYPE, dtype: int64


In [97]:
combo_num = 62

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Physiology 

TREATS_DVtPS              337
USES_PSuDV                218
CAUSES_DVcPS               87
AFFECTS_DVafPS             67
PREVENTS_DVpvPS            23
NEG_AFFECTS_DVnafPS        12
PROCESS_OF_PSpro>DV        11
NEG_TREATS_DVntPS          10
STIMULATES_DVstPS           9
NEG_USES_PSnuDV             6
LOCATION_OF_PSloDV          6
PRODUCES_PSpd>DV            4
PROCESS_OF_DVpro>PS         3
DISRUPTS_DVdsPS             3
AUGMENTS_DVagPS             3
PRECEDES_DVpc>PS            3
ASSOCIATED_WITH_PSawDV      2
compared_with_DVcpwPS       2
NEG_STIMULATES_DVnstPS      1
NEG_CAUSES_DVncPS           1
DIAGNOSES_DVdgPS            1
ADMINISTERED_TO_DVatPS      1
COEXISTS_WITH_PScwDV        1
NEG_DISRUPTS_DVndsPS        1
INHIBITS_DVinPS             1
Name: TYPE, dtype: int64


In [98]:
combo_num = 63

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Devices --- Procedures 

USES_PRuDV                   75829
compared_with_PRcpwDV         6224
NEG_USES_PRnuDV               1187
higher_than_PRhtDV            1146
METHOD_OF_PRmoDV               625
ISA_DVi>PR                     531
COEXISTS_WITH_DVcwPR           354
same_as_DVsaPR                 240
lower_than_DVltPR              200
TREATS_DVtPR                   128
PRECEDES_PRpc>DV               114
PRECEDES_DVpc>PR               100
NEG_higher_than_DVnhtPR         72
ISA_PRi>DV                      46
LOCATION_OF_DVloPR              44
NEG_METHOD_OF_DVnmoPR           20
different_from_DVdfPR           20
AFFECTS_PRafDV                  10
NEG_lower_than_DVnltPR           7
different_than_DVdtPR            6
NEG_COEXISTS_WITH_DVncwPR        4
DIAGNOSES_PRdgDV                 3
CAUSES_DVcPR                     2
PROCESS_OF_DVpro>PR              2
NEG_TREATS_DVntPR                1
NEG_PRECEDES_DVnpcPR             1
PREVENTS_DVpvPR                  1
Name: TYPE, dtype: int64


In [99]:
to_merge = ['USES_PRuDV', 'METHOD_OF_PRmoDV']
merge_edge_types(to_merge, 'USES_PRuDV')

In [100]:
combo_num = 64

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Disorders 

COEXISTS_WITH_DOcwDO             619157
CAUSES_DOcDO                     206444
ISA_DOi>DO                        95431
AFFECTS_DOafDO                    95344
PREDISPOSES_DOpsDO                69291
ASSOCIATED_WITH_DOawDO            33876
COMPLICATES_DOcpDO                30032
MANIFESTATION_OF_DOmfo>DO         29595
PRECEDES_DOpc>DO                  19734
NEG_COEXISTS_WITH_DOncwDO         16998
OCCURS_IN_DOoiDO                  16651
LOCATION_OF_DOloDO                14641
AUGMENTS_DOagDO                    8309
PROCESS_OF_DOpro>DO                7345
TREATS_DOtDO                       6050
PART_OF_DOpo>DO                    4840
NEG_AFFECTS_DOnafDO                4443
NEG_CAUSES_DOncDO                  4398
DIAGNOSES_DOdgDO                   3996
NEG_PREDISPOSES_DOnpsDO            3117
NEG_ASSOCIATED_WITH_DOnawDO        1171
PREVENTS_DOpvDO                     899
NEG_MANIFESTATION_OF_DOnmfoDO       359
NEG_OCCURS_IN_DOnoiDO               280
NEG_AUGMENTS_D

In [101]:
to_merge = ['COEXISTS_WITH_DOcwDO', 'ASSOCIATED_WITH_DOawDO', 'AFFECTS_DOafDO', 'OCCURS_IN_DOoiDO', 'LOCATION_OF_DOloDO']
merge_edge_types(to_merge, 'ASSOCIATED_WITH_DOawDO')
    
# This should be directed, one disease leading to or somehow making worse another
to_merge = ['CAUSES_DOcDO', 'PREDISPOSES_DOpsDO','COMPLICATES_DOcpDO', 'PRECEDES_DOpc>DO', 'AUGMENTS_DOagDO']
merge_edge_types(to_merge, 'AUGMENTS_DOag>DO')
 
to_merge = ['ISA_DOi>DO', 'MANIFESTATION_OF_DOmfo>DO']
merge_edge_types(to_merge, 'ISA_DOi>DO')
 
to_drop = ['NEG_COEXISTS_WITH_DOncwDO']
drop_edges_from_list(to_drop)

In [102]:
combo_num = 65

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Genes & Molecular Sequences 

ASSOCIATED_WITH_GawDO           258028
TREATS_GtDO                      86382
CAUSES_GcDO                      81729
AFFECTS_GafDO                    65326
PREDISPOSES_GpsDO                47897
PART_OF_Gpo>DO                   43870
AUGMENTS_GagDO                   37256
DISRUPTS_GdsDO                   30451
PRODUCES_DOpd>G                  20970
PREVENTS_GpvDO                   13952
NEG_ASSOCIATED_WITH_GnawDO        7265
NEG_AFFECTS_GnafDO                4215
LOCATION_OF_GloDO                 2923
NEG_CAUSES_GncDO                  2653
NEG_PREDISPOSES_GnpsDO            1597
NEG_TREATS_GntDO                  1557
NEG_PRODUCES_DOnpdG                951
NEG_AUGMENTS_GnagDO                882
COMPLICATES_GcpDO                  814
NEG_PART_OF_GnpoDO                 683
NEG_DISRUPTS_GndsDO                682
NEG_PREVENTS_GnpvDO                437
INHIBITS_GinDO                     399
USES_DOuG                          132
COEXISTS_WITH_DOcwG 

In [103]:
# negative association wtih disorder
to_merge = ['TREATS_GtDO', 'PREVENTS_GpvDO','DISRUPTS_GdsDO', 'INHIBITS_GinDO']
merge_edge_types(to_merge, 'INHIBITS_GinDO')

# positive association with disorder
to_merge = ['CAUSES_GcDO', 'PREDISPOSES_GpsDO', 'AUGMENTS_GagDO','COMPLICATES_GcpDO']
merge_edge_types(to_merge, 'AUGMENTS_GagDO')

# General association with disorder
to_merge = ['ASSOCIATED_WITH_GawDO', 'AFFECTS_GafDO','PART_OF_Gpo>DO']
merge_edge_types(to_merge, 'ASSOCIATED_WITH_GawDO')
change_edge_type('PRODUCES_DOpd>G', 'ASSOCIATED_WITH_GawDO', swap=True)

In [104]:
combo_num = 66

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Geographic Areas 

COEXISTS_WITH_GAcwDO     3
LOCATION_OF_GAloDO       2
NEG_OCCURS_IN_DOnoiGA    1
AFFECTS_GAafDO           1
TREATS_GAtDO             1
DIAGNOSES_GAdgDO         1
Name: TYPE, dtype: int64


In [105]:
combo_num = 67

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Living Beings 

PROCESS_OF_DOpro>LB            577499
AFFECTS_DOafLB                  56015
CAUSES_LBcDO                    51965
OCCURS_IN_DOoiLB                45228
PART_OF_DOpo>LB                 25065
NEG_PROCESS_OF_DOnproLB         22448
DIAGNOSES_LBdgDO                17415
LOCATION_OF_LBloDO              13185
COEXISTS_WITH_LBcwDO             7015
INTERACTS_WITH_LBiwDO            4461
NEG_AFFECTS_DOnafLB              2646
NEG_OCCURS_IN_DOnoiLB            1595
TREATS_DOtLB                     1161
ASSOCIATED_WITH_LBawDO            956
NEG_CAUSES_LBncDO                 900
ADMINISTERED_TO_DOatLB            536
NEG_DIAGNOSES_LBndgDO             464
NEG_PART_OF_DOnpoLB               453
NEG_LOCATION_OF_LBnloDO           299
NEG_INTERACTS_WITH_LBniwDO        234
NEG_COEXISTS_WITH_LBncwDO         201
PREDISPOSES_LBpsDO                166
PART_OF_LBpo>DO                    69
NEG_TREATS_DOntLB                  55
PROCESS_OF_LBpro>DO                41
NEG_ADMINISTERED_TO_

In [106]:
# Either a living being had a disease
# Or causes a disease
to_merge = ['CAUSES_LBcDO', 'ASSOCIATED_WITH_LBawDO']
merge_edge_types(to_merge, 'CAUSES_LBcDO')
    
to_merge = ['AFFECTS_DOafLB',  'OCCURS_IN_DOoiLB', 'PART_OF_DOpo>LB', 'PROCESS_OF_DOpro>LB']
merge_edge_types(to_merge, 'AFFECTS_DOafLB')
to_merge = ['LOCATION_OF_LBloDO','COEXISTS_WITH_LBcwDO']
merge_edge_types(to_merge, 'AFFECTS_DOafLB', swap=True)

# Diagonoses likely unhelpful
to_drop = ['NEG_PROCESS_OF_DOnproLB', 'DIAGNOSES_LBdgDO']
drop_edges_from_list(to_drop)

In [107]:
combo_num = 68

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Objects 

CAUSES_OBcDO                   10596
AFFECTS_OBafDO                  4207
TREATS_OBtDO                    3771
PREDISPOSES_OBpsDO              3310
ASSOCIATED_WITH_OBawDO           997
PREVENTS_OBpvDO                  569
NEG_CAUSES_OBncDO                377
NEG_AFFECTS_OBnafDO              298
AUGMENTS_OBagDO                  209
DISRUPTS_OBdsDO                  209
NEG_PREDISPOSES_OBnpsDO          159
USES_DOuOB                       111
NEG_TREATS_OBntDO                 80
LOCATION_OF_OBloDO                54
NEG_ASSOCIATED_WITH_OBnawDO       33
INHIBITS_OBinDO                   28
NEG_PREVENTS_OBnpvDO              26
DIAGNOSES_OBdgDO                  14
PROCESS_OF_DOpro>OB               11
PRODUCES_DOpd>OB                   9
PART_OF_OBpo>DO                    7
PART_OF_DOpo>OB                    6
COEXISTS_WITH_DOcwOB               6
NEG_AUGMENTS_OBnagDO               5
COMPLICATES_OBcpDO                 4
NEG_DISRUPTS_OBndsDO               3
NEG_DIAGNOSES_

In [108]:
combo_num = 69

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Occupations 

TREATS_OCtDO                 557
PROCESS_OF_DOpro>OC          140
COEXISTS_WITH_OCcwDO         137
AFFECTS_DOafOC               125
ASSOCIATED_WITH_DOawOC        99
CAUSES_OCcDO                  52
PREVENTS_OCpvDO               50
METHOD_OF_DOmoOC              36
MANIFESTATION_OF_OCmfo>DO     19
DIAGNOSES_OCdgDO              18
PREDISPOSES_OCpsDO            12
MANIFESTATION_OF_DOmfo>OC     12
NEG_TREATS_OCntDO             12
OCCURS_IN_DOoiOC              11
PRECEDES_OCpc>DO               5
PROCESS_OF_OCpro>DO            5
PRECEDES_DOpc>OC               4
PART_OF_DOpo>OC                3
INTERACTS_WITH_DOiwOC          3
NEG_PREDISPOSES_OCnpsDO        2
NEG_PREVENTS_OCnpvDO           2
NEG_PROCESS_OF_DOnproOC        1
ADMINISTERED_TO_OCatDO         1
NEG_DIAGNOSES_OCndgDO          1
NEG_METHOD_OF_DOnmoOC          1
Name: TYPE, dtype: int64


In [109]:
combo_num = 70

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Organizations 

LOCATION_OF_ORloDO         161
TREATS_ORtDO                32
PROCESS_OF_DOpro>OR         14
AFFECTS_ORafDO               6
DIAGNOSES_ORdgDO             5
PREVENTS_ORpvDO              5
NEG_LOCATION_OF_ORnloDO      2
COEXISTS_WITH_DOcwOR         2
CAUSES_ORcDO                 2
NEG_TREATS_ORntDO            2
COMPLICATES_ORcpDO           1
PART_OF_DOpo>OR              1
NEG_AFFECTS_ORnafDO          1
PART_OF_ORpo>DO              1
PRODUCES_DOpd>OR             1
Name: TYPE, dtype: int64


In [110]:
combo_num = 71

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Phenomena 

AFFECTS_PHafDO                   14351
PROCESS_OF_DOpro>PH              11452
CAUSES_PHcDO                      5865
PROCESS_OF_PHpro>DO               3132
COEXISTS_WITH_PHcwDO              2661
PREDISPOSES_PHpsDO                1607
ASSOCIATED_WITH_PHawDO            1311
NEG_AFFECTS_PHnafDO                905
ISA_DOi>PH                         381
NEG_PROCESS_OF_DOnproPH            338
DIAGNOSES_PHdgDO                   321
MANIFESTATION_OF_PHmfo>DO          228
MANIFESTATION_OF_DOmfo>PH          180
TREATS_PHtDO                       157
NEG_CAUSES_PHncDO                  156
NEG_COEXISTS_WITH_DOncwPH          117
LOCATION_OF_PHloDO                  99
NEG_PREDISPOSES_PHnpsDO             76
ISA_PHi>DO                          73
DISRUPTS_DOdsPH                     56
NEG_ASSOCIATED_WITH_PHnawDO         30
AUGMENTS_PHagDO                     25
PREVENTS_PHpvDO                     24
PRECEDES_DOpc>PH                    18
COMPLICATES_DOcpPH                  17

In [111]:
to_merge = ['AFFECTS_PHafDO','CAUSES_PHcDO','PROCESS_OF_PHpro>DO','COEXISTS_WITH_PHcwDO','ASSOCIATED_WITH_PHawDO','PREDISPOSES_PHpsDO']
merge_edge_types(to_merge, 'AFFECTS_PHafDO')

change_edge_type('PROCESS_OF_DOpro>PH', 'AFFECTS_PHafDO', swap=True)

In [112]:
combo_num = 72

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Physiology 

AFFECTS_PSafDO                   86493
ASSOCIATED_WITH_PSawDO           51541
PROCESS_OF_DOpro>PS              32973
CAUSES_DOcPS                     19530
COEXISTS_WITH_PScwDO             12941
MANIFESTATION_OF_DOmfo>PS         8648
NEG_AFFECTS_PSnafDO               5790
PREDISPOSES_PSpsDO                4882
PROCESS_OF_PSpro>DO               2942
NEG_ASSOCIATED_WITH_PSnawDO       2472
PRECEDES_DOpc>PS                  1780
DISRUPTS_DOdsPS                   1777
TREATS_PStDO                      1600
NEG_PROCESS_OF_DOnproPS           1275
INTERACTS_WITH_DOiwPS              931
DIAGNOSES_PSdgDO                   913
NEG_CAUSES_DOncPS                  777
OCCURS_IN_DOoiPS                   471
NEG_COEXISTS_WITH_DOncwPS          426
ISA_DOi>PS                         250
NEG_PREDISPOSES_PSnpsDO            239
PREVENTS_PSpvDO                    188
NEG_MANIFESTATION_OF_DOnmfoPS      173
LOCATION_OF_PSloDO                 159
NEG_INTERACTS_WITH_PSniwDO         15

In [113]:
to_merge = ['AFFECTS_PSafDO','ASSOCIATED_WITH_PSawDO','COEXISTS_WITH_PScwDO',
             'PREDISPOSES_PSpsDO','TREATS_PStDO']
merge_edge_types(to_merge, 'AFFECTS_PSafDO')

to_merge = ['PROCESS_OF_DOpro>PS','CAUSES_DOcPS','MANIFESTATION_OF_DOmfo>PS','PRECEDES_DOpc>PS']
merge_edge_types(to_merge, 'CAUSES_DOcPS')

In [114]:
combo_num = 73

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Disorders --- Procedures 

TREATS_PRtDO                   425717
DIAGNOSES_PRdgDO               223926
AFFECTS_PRafDO                  57701
CAUSES_PRcDO                    52979
PREVENTS_PRpvDO                 51422
PREDISPOSES_PRpsDO              17156
NEG_TREATS_PRntDO               15447
NEG_DIAGNOSES_PRndgDO            6356
NEG_AFFECTS_PRnafDO              4333
PRECEDES_PRpc>DO                 4220
NEG_PREVENTS_PRnpvDO             2476
COEXISTS_WITH_PRcwDO             2333
NEG_CAUSES_PRncDO                2215
METHOD_OF_DOmoPR                 1985
COMPLICATES_PRcpDO               1546
ISA_PRi>DO                       1515
NEG_PREDISPOSES_PRnpsDO          1423
USES_DOuPR                       1352
PROCESS_OF_DOpro>PR               898
LOCATION_OF_DOloPR                591
ADMINISTERED_TO_PRatDO            348
ASSOCIATED_WITH_PRawDO            240
NEG_COEXISTS_WITH_PRncwDO         136
STIMULATES_PRstDO                 116
NEG_USES_DOnuPR                   111
PRECEDES_DOpc>PR       

In [115]:
to_merge = ['TREATS_PRtDO','PREVENTS_PRpvDO']
merge_edge_types(to_merge, 'TREATS_PRtDO')

# keep 'DIAGNOSES_PRdgDO' asis
# keep 'AFFECTS_PRafDO' asis

to_merge = ['CAUSES_PRcDO', 'PREDISPOSES_PRpsDO','PRECEDES_PRpc>DO','COMPLICATES_PRcpDO']
merge_edge_types(to_merge, 'PREDISPOSES_PRpsDO')

to_drop = ['NEG_TREATS_PRntDO', 'NEG_DIAGNOSES_PRndgDO', 'NEG_AFFECTS_PRnafDO']
drop_edges_from_list(to_drop)

In [116]:
combo_num = 74

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Genes & Molecular Sequences 

INTERACTS_WITH_GiwG         161726
STIMULATES_GstG             123328
COEXISTS_WITH_GcwG          115029
INHIBITS_GinG                80164
PRODUCES_Gpd>G               43665
compared_with_GcpwG          36955
PART_OF_Gpo>G                26604
higher_than_GhtG             11001
NEG_INTERACTS_WITH_GniwG     10150
CONVERTS_TO_Gct>G             5239
ISA_Gi>G                      5204
NEG_COEXISTS_WITH_GncwG       4648
NEG_STIMULATES_GnstG          4628
NEG_INHIBITS_GninG            2248
lower_than_GltG               2144
NEG_PRODUCES_GnpdG            2098
same_as_GsaG                  2051
USES_GuG                       945
different_from_GdfG            857
NEG_PART_OF_GnpoG              826
NEG_higher_than_GnhtG          256
different_than_GdtG            215
ASSOCIATED_WITH_GawG            64
AFFECTS_GafG                    63
NEG_CONVERTS_TO_GnctG           52
NEG_lower_than_GnltG            44
PRECEDES_Gpc>G              

In [117]:
to_merge = ['INTERACTS_WITH_GiwG','COEXISTS_WITH_GcwG']
merge_edge_types(to_merge, 'INTERACTS_WITH_GiwG')

to_merge = ['STIMULATES_GstG','PRODUCES_Gpd>G']
merge_edge_types(to_merge, 'STIMULATES_Gst>G')

to_merge = ['CONVERTS_TO_Gct>G', 'ISA_Gi>G']
merge_edge_types(to_merge, 'ISA_Gi>G')

# keep 'INHIBITS_GinG' as is

drop_edges_from_list(['compared_with_GcpwG'])

In [118]:
combo_num = 75

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Geographic Areas 

COEXISTS_WITH_GcwGA     2
INTERACTS_WITH_GAiwG    1
Name: TYPE, dtype: int64


In [119]:
combo_num = 76

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Living Beings 

LOCATION_OF_LBloG             185404
PART_OF_Gpo>LB                141133
INTERACTS_WITH_GiwLB            7232
TREATS_GtLB                     6002
ADMINISTERED_TO_GatLB           5661
NEG_LOCATION_OF_LBnloG          3920
PRODUCES_LBpd>G                 2660
NEG_PART_OF_GnpoLB              1771
AFFECTS_GafLB                    624
PROCESS_OF_Gpro>LB               304
NEG_TREATS_GntLB                 249
NEG_INTERACTS_WITH_GniwLB        214
AUGMENTS_GagLB                   167
NEG_ADMINISTERED_TO_GnatLB       145
COEXISTS_WITH_LBcwG              139
DISRUPTS_GdsLB                   112
PREVENTS_GpvLB                    54
NEG_PRODUCES_LBnpdG               54
STIMULATES_GstLB                  49
PART_OF_LBpo>G                    30
PRODUCES_Gpd>LB                   26
compared_with_GcpwLB              22
NEG_AFFECTS_GnafLB                22
INHIBITS_GinLB                    19
CAUSES_GcLB                       11
USES_GuLB                  

In [120]:
# Merge to Part_of that is non-directional
change_edge_type('LOCATION_OF_LBloG', 'PART_OF_GpoLB', swap=True)
change_edge_type('PART_OF_Gpo>LB', 'PART_OF_GpoLB')

In [121]:
combo_num = 77

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Objects 

compared_with_OBcpwG         350
INTERACTS_WITH_GiwOB         298
LOCATION_OF_OBloG            287
COEXISTS_WITH_GcwOB          262
STIMULATES_GstOB             176
INHIBITS_OBinG               112
ISA_Gi>OB                     78
higher_than_GhtOB             65
USES_OBuG                     64
PRODUCES_Gpd>OB               37
NEG_INTERACTS_WITH_OBniwG     28
PART_OF_OBpo>G                22
lower_than_OBltG              18
same_as_GsaOB                 11
ASSOCIATED_WITH_GawOB         10
NEG_COEXISTS_WITH_GncwOB       8
NEG_INHIBITS_OBninG            7
PRODUCES_OBpd>G                5
CONVERTS_TO_OBct>G             5
ISA_OBi>G                      4
AUGMENTS_GagOB                 3
different_than_OBdtG           2
CONVERTS_TO_Gct>OB             2
AFFECTS_OBafG                  2
NEG_STIMULATES_OBnstG          2
PART_OF_Gpo>OB                 2
TREATS_OBtG                    1
NEG_LOCATION_OF_OBnloG         1
PRECEDES_OBpc>G                1
d

In [122]:
combo_num = 78

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Occupations 

AFFECTS_GafOC            116
ASSOCIATED_WITH_GawOC     23
METHOD_OF_GmoOC           18
DISRUPTS_GdsOC            17
AUGMENTS_GagOC             9
CAUSES_GcOC                4
USES_OCuG                  4
NEG_AFFECTS_GnafOC         3
PREDISPOSES_GpsOC          2
STIMULATES_OCstG           1
NEG_USES_OCnuG             1
LOCATION_OF_OCloG          1
Name: TYPE, dtype: int64


In [123]:
combo_num = 79

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Organizations 

LOCATION_OF_ORloG       15
INHIBITS_GinOR           5
PART_OF_Gpo>OR           5
TREATS_GtOR              2
compared_with_GcpwOR     2
USES_ORuG                1
CONVERTS_TO_Gct>OR       1
STIMULATES_ORstG         1
AUGMENTS_GagOR           1
PRODUCES_ORpd>G          1
Name: TYPE, dtype: int64


In [124]:
combo_num = 80

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Phenomena 

AFFECTS_GafPH                 13098
AUGMENTS_GagPH                 5919
DISRUPTS_GdsPH                 3321
CAUSES_GcPH                    1491
NEG_AFFECTS_GnafPH              798
LOCATION_OF_PHloG               414
PRODUCES_PHpd>G                 376
PART_OF_Gpo>PH                  286
NEG_AUGMENTS_GnagPH             217
INTERACTS_WITH_GiwPH            196
NEG_DISRUPTS_GndsPH              88
USES_PHuG                        87
COEXISTS_WITH_GcwPH              68
STIMULATES_PHstG                 61
INHIBITS_PHinG                   57
TREATS_GtPH                      43
NEG_CAUSES_GncPH                 42
ASSOCIATED_WITH_GawPH            24
NEG_PRODUCES_PHnpdG              15
NEG_INTERACTS_WITH_PHniwG        10
compared_with_PHcpwG             10
NEG_LOCATION_OF_PHnloG           10
PROCESS_OF_PHpro>G                8
ADMINISTERED_TO_GatPH             8
PRODUCES_Gpd>PH                   8
PROCESS_OF_Gpro>PH                5
higher_than_GhtPH   

In [125]:
to_merge = ['AFFECTS_GafPH', 'AUGMENTS_GagPH', 'DISRUPTS_GdsPH', 'CAUSES_GcPH']
merge_edge_types(to_merge, 'AFFECTS_GafPH')

In [126]:
combo_num = 81

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Physiology 

AFFECTS_GafPS                   174080
AUGMENTS_GagPS                   76441
DISRUPTS_GdsPS                   54395
CAUSES_GcPS                      32617
NEG_AFFECTS_GnafPS               12246
PRODUCES_PSpd>G                   7077
NEG_AUGMENTS_GnagPS               2867
NEG_DISRUPTS_GndsPS               2269
NEG_CAUSES_GncPS                  1368
LOCATION_OF_PSloG                  973
ASSOCIATED_WITH_GawPS              845
INHIBITS_GinPS                     509
PART_OF_Gpo>PS                     467
TREATS_GtPS                        227
INTERACTS_WITH_PSiwG               222
STIMULATES_GstPS                   169
NEG_PRODUCES_PSnpdG                168
COEXISTS_WITH_PScwG                163
PREVENTS_GpvPS                      86
PREDISPOSES_GpsPS                   77
COMPLICATES_GcpPS                   49
PROCESS_OF_PSpro>G                  48
NEG_ASSOCIATED_WITH_GnawPS          42
NEG_LOCATION_OF_PSnloG              36
compared_with_GcpwP

In [127]:
to_merge = ['AFFECTS_GafPS', 'AUGMENTS_GagPS', 'DISRUPTS_GdsPS', 'CAUSES_GcPS']
merge_edge_types(to_merge, 'AFFECTS_GafPS')

 
drop_edges_from_list(['NEG_AFFECTS_GnafPS'])

In [128]:
combo_num = 82

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Genes & Molecular Sequences --- Procedures 

USES_PRuG                    73378
compared_with_PRcpwG          9578
ISA_Gi>PR                     2384
higher_than_PRhtG             2171
METHOD_OF_GmoPR               1096
NEG_USES_PRnuG                 942
AFFECTS_GafPR                  846
AUGMENTS_GagPR                 557
PRECEDES_Gpc>PR                458
same_as_PRsaG                  366
DISRUPTS_GdsPR                 349
lower_than_PRltG               331
COEXISTS_WITH_PRcwG            288
PRODUCES_PRpd>G                245
PRECEDES_PRpc>G                240
CAUSES_GcPR                    213
LOCATION_OF_PRloG              121
NEG_AFFECTS_GnafPR             112
TREATS_PRtG                    107
INTERACTS_WITH_PRiwG            91
NEG_higher_than_PRnhtG          83
PART_OF_Gpo>PR                  83
INHIBITS_GinPR                  69
different_from_PRdfG            56
STIMULATES_GstPR                55
ISA_PRi>G                       45
DIAGNOSES_PRdgG                 20
different_

In [129]:
# This is the only real edge in here: 'USES_PRuG'
# However method_of can be added to it
change_edge_type('METHOD_OF_GmoPR', 'USES_PRuG', swap=True)

In [130]:
combo_num = 83

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Geographic Areas --- Geographic Areas 

ISA_GAi>GA    1999
Name: TYPE, dtype: int64


In [131]:
combo_num = 84

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Geographic Areas --- Living Beings 

ISA_GAi>LB            3
LOCATION_OF_LBloGA    1
TREATS_GAtLB          1
ISA_LBi>GA            1
Name: TYPE, dtype: int64


In [132]:
combo_num = 85

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Geographic Areas --- Phenomena 

ISA_PHi>GA        1
AFFECTS_GAafPH    1
Name: TYPE, dtype: int64


In [133]:
combo_num = 86

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Geographic Areas --- Physiology 

PROCESS_OF_PSpro>GA       1
AUGMENTS_GAagPS           1
ASSOCIATED_WITH_PSawGA    1
AFFECTS_GAafPS            1
Name: TYPE, dtype: int64


In [134]:
combo_num = 87

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Geographic Areas --- Procedures 

LOCATION_OF_GAloPR    5
METHOD_OF_GAmoPR      2
USES_PRuGA            1
Name: TYPE, dtype: int64


In [135]:
combo_num = 88

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Living Beings 

ISA_LBi>LB                    73724
INTERACTS_WITH_LBiwLB         38429
PROCESS_OF_LBpro>LB           17331
COEXISTS_WITH_LBcwLB           3518
TREATS_LBtLB                   1917
NEG_INTERACTS_WITH_LBniwLB     1448
PART_OF_LBpo>LB                1397
LOCATION_OF_LBloLB             1288
NEG_PROCESS_OF_LBnproLB         351
NEG_COEXISTS_WITH_LBncwLB        94
NEG_TREATS_LBntLB                87
ADMINISTERED_TO_LBatLB           48
NEG_LOCATION_OF_LBnloLB          31
AFFECTS_LBafLB                   28
NEG_PART_OF_LBnpoLB              27
USES_LBuLB                       14
compared_with_LBcpwLB            11
OCCURS_IN_LBoiLB                  6
CAUSES_LBcLB                      5
higher_than_LBhtLB                3
DIAGNOSES_LBdgLB                  3
lower_than_LBltLB                 1
AUGMENTS_LBagLB                   1
same_as_LBsaLB                    1
NEG_higher_than_LBnhtLB           1
ASSOCIATED_WITH_LBawLB            1
PREVENTS_LBpvLB               

In [136]:
to_merge = ['ISA_LBi>LB','PART_OF_LBpo>LB']
merge_edge_types(to_merge, 'ISA_LBi>LB')

to_merge = ['INTERACTS_WITH_LBiwLB','COEXISTS_WITH_LBcwLB']
merge_edge_types(to_merge, 'INTERACTS_WITH_LBiwLB')

In [137]:
combo_num = 89

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Objects 

TREATS_OBtLB                   1447
LOCATION_OF_LBloOB              966
USES_LBuOB                      744
PART_OF_OBpo>LB                 672
ADMINISTERED_TO_OBatLB          479
DIAGNOSES_OBdgLB                 58
NEG_ADMINISTERED_TO_OBnatLB      44
PRODUCES_LBpd>OB                 43
NEG_TREATS_OBntLB                41
NEG_USES_LBnuOB                  18
NEG_LOCATION_OF_LBnloOB          17
PROCESS_OF_OBpro>LB              16
NEG_PART_OF_OBnpoLB              14
ISA_OBi>LB                       10
INTERACTS_WITH_LBiwOB             9
ISA_LBi>OB                        6
AFFECTS_OBafLB                    4
PREVENTS_OBpvLB                   2
ASSOCIATED_WITH_OBawLB            2
OCCURS_IN_OBoiLB                  1
METHOD_OF_OBmoLB                  1
NEG_PROCESS_OF_OBnproLB           1
DISRUPTS_OBdsLB                   1
AUGMENTS_OBagLB                   1
Name: TYPE, dtype: int64


In [138]:
combo_num = 90

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Occupations 

PROCESS_OF_OCpro>LB            286
TREATS_OCtLB                   224
ADMINISTERED_TO_OCatLB          90
AFFECTS_OCafLB                  28
INTERACTS_WITH_OCiwLB           16
NEG_TREATS_OCntLB               14
NEG_ADMINISTERED_TO_OCnatLB     13
ISA_LBi>OC                       7
ISA_OCi>LB                       7
DIAGNOSES_LBdgOC                 4
METHOD_OF_LBmoOC                 4
PROCESS_OF_LBpro>OC              2
OCCURS_IN_OCoiLB                 2
NEG_PROCESS_OF_OCnproLB          1
NEG_INTERACTS_WITH_LBniwOC       1
Name: TYPE, dtype: int64


In [139]:
combo_num = 91

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Organizations 

TREATS_ORtLB                  20
LOCATION_OF_ORloLB            16
PROCESS_OF_ORpro>LB            7
ADMINISTERED_TO_ORatLB         4
PART_OF_ORpo>LB                2
INTERACTS_WITH_ORiwLB          1
NEG_INTERACTS_WITH_ORniwLB     1
NEG_TREATS_ORntLB              1
USES_LBuOR                     1
Name: TYPE, dtype: int64


In [140]:
combo_num = 92

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Phenomena 

PROCESS_OF_PHpro>LB        10143
AFFECTS_PHafLB              1238
OCCURS_IN_PHoiLB             516
NEG_PROCESS_OF_PHnproLB      248
LOCATION_OF_LBloPH           218
TREATS_PHtLB                  67
NEG_AFFECTS_PHnafLB           62
PART_OF_PHpo>LB               46
NEG_OCCURS_IN_PHnoiLB         24
ADMINISTERED_TO_PHatLB        20
CAUSES_LBcPH                  17
DIAGNOSES_PHdgLB              12
NEG_LOCATION_OF_LBnloPH       11
INTERACTS_WITH_LBiwPH          8
PROCESS_OF_LBpro>PH            7
DISRUPTS_LBdsPH                4
PRODUCES_PHpd>LB               4
COEXISTS_WITH_PHcwLB           3
PREDISPOSES_PHpsLB             2
USES_LBuPH                     2
NEG_CAUSES_LBncPH              1
NEG_TREATS_PHntLB              1
NEG_DIAGNOSES_PHndgLB          1
AUGMENTS_LBagPH                1
PART_OF_LBpo>PH                1
Name: TYPE, dtype: int64


In [141]:
combo_num = 93

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Physiology 

PROCESS_OF_PSpro>LB            146805
AFFECTS_PSafLB                  18044
NEG_PROCESS_OF_PSnproLB          3954
NEG_AFFECTS_PSnafLB              1039
CAUSES_LBcPS                     1026
OCCURS_IN_PSoiLB                  816
PRODUCES_PSpd>LB                  318
TREATS_PStLB                      260
INTERACTS_WITH_PSiwLB             241
ADMINISTERED_TO_PSatLB            144
LOCATION_OF_LBloPS                102
DIAGNOSES_LBdgPS                   99
PROCESS_OF_LBpro>PS                88
PART_OF_PSpo>LB                    63
NEG_OCCURS_IN_PSnoiLB              43
NEG_CAUSES_LBncPS                  32
AUGMENTS_LBagPS                    27
DISRUPTS_LBdsPS                    21
ASSOCIATED_WITH_PSawLB             20
ISA_LBi>PS                         20
ISA_PSi>LB                         17
NEG_INTERACTS_WITH_PSniwLB         13
NEG_ADMINISTERED_TO_PSnatLB        10
NEG_PRODUCES_PSnpdLB               10
MANIFESTATION_OF_LBmfo>PS           9
NEG_TREATS_PSntLB  

In [142]:
change_edge_type('PROCESS_OF_PSpr>LB', 'PROCESS_OF_PSprLB')
change_edge_type('CAUSES_LBcPS', 'PROCESS_OF_PSprLB', swap=True)

to_merge = ['AFFECTS_PSafLB', 'OCCURS_IN_PSoiLB','PRODUCES_PSpd>LB']
merge_edge_types(to_merge, 'AFFECTS_PSafLB')

In [143]:
combo_num = 94

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Living Beings --- Procedures 

TREATS_PRtLB                   98453
ADMINISTERED_TO_PRatLB         54764
DIAGNOSES_PRdgLB                6274
NEG_TREATS_PRntLB               4212
NEG_ADMINISTERED_TO_PRnatLB     3742
AFFECTS_PRafLB                  3097
PROCESS_OF_PRpro>LB             1302
PREVENTS_PRpvLB                  502
USES_LBuPR                       402
NEG_AFFECTS_PRnafLB              325
LOCATION_OF_LBloPR               274
METHOD_OF_PRmoLB                 193
NEG_DIAGNOSES_PRndgLB            171
PART_OF_PRpo>LB                  107
NEG_PROCESS_OF_PRnproLB           66
INTERACTS_WITH_PRiwLB             25
compared_with_PRcpwLB             18
NEG_PREVENTS_PRnpvLB              16
NEG_USES_LBnuPR                   14
CAUSES_PRcLB                      12
OCCURS_IN_PRoiLB                   9
COEXISTS_WITH_PRcwLB               9
PROCESS_OF_LBpro>PR                5
PRODUCES_LBpd>PR                   5
NEG_LOCATION_OF_LBnloPR            4
ISA_LBi>PR                         4
higher_

In [144]:
# keep 'TREATS_PRtLB' edge as-is
 
to_merge = ['ADMINISTERED_TO_PRatLB', 'AFFECTS_PRafLB']
merge_edge_types(to_merge, 'ADMINISTERED_TO_PRatLB')

In [145]:
combo_num = 95

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Objects 

ISA_OBi>OB                   2807
USES_OBuOB                   1214
compared_with_OBcpwOB         195
higher_than_OBhtOB             78
COEXISTS_WITH_OBcwOB           77
INTERACTS_WITH_OBiwOB          58
STIMULATES_OBstOB              21
same_as_OBsaOB                 20
lower_than_OBltOB              17
INHIBITS_OBinOB                15
NEG_USES_OBnuOB                15
NEG_higher_than_OBnhtOB        11
LOCATION_OF_OBloOB              8
different_from_OBdfOB           5
CONVERTS_TO_OBct>OB             5
NEG_COEXISTS_WITH_OBncwOB       4
different_than_OBdtOB           3
PRODUCES_OBpd>OB                1
NEG_INHIBITS_OBninOB            1
CAUSES_OBcOB                    1
TREATS_OBtOB                    1
NEG_lower_than_OBnltOB          1
Name: TYPE, dtype: int64


In [146]:
combo_num = 96

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Occupations 

METHOD_OF_OBmoOC       27
AFFECTS_OBafOC          5
TREATS_OBtOC            1
NEG_AFFECTS_OBnafOC     1
Name: TYPE, dtype: int64


In [147]:
combo_num = 97

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Organizations 

USES_OBuOR            112
LOCATION_OF_ORloOB     65
ISA_OBi>OR             27
ISA_ORi>OB             10
NEG_USES_OBnuOR         1
Name: TYPE, dtype: int64


In [148]:
combo_num = 98

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Phenomena 

AFFECTS_OBafPH            437
AUGMENTS_OBagPH            46
DISRUPTS_OBdsPH            29
NEG_AFFECTS_OBnafPH        28
USES_PHuOB                 16
TREATS_OBtPH                7
CAUSES_OBcPH                7
LOCATION_OF_OBloPH          5
PROCESS_OF_PHpro>OB         2
COEXISTS_WITH_OBcwPH        2
NEG_AUGMENTS_OBnagPH        2
NEG_USES_PHnuOB             2
INHIBITS_OBinPH             1
PREDISPOSES_OBpsPH          1
compared_with_PHcpwOB       1
ISA_OBi>PH                  1
PRODUCES_OBpd>PH            1
ASSOCIATED_WITH_OBawPH      1
ISA_PHi>OB                  1
Name: TYPE, dtype: int64


In [149]:
combo_num = 99

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Physiology 

AFFECTS_OBafPS                 3743
NEG_AFFECTS_OBnafPS             416
AUGMENTS_OBagPS                 248
DISRUPTS_OBdsPS                 144
CAUSES_OBcPS                     74
USES_PSuOB                       46
TREATS_OBtPS                     31
NEG_AUGMENTS_OBnagPS             14
INHIBITS_OBinPS                  14
NEG_DISRUPTS_OBndsPS             13
ASSOCIATED_WITH_PSawOB           13
PRODUCES_PSpd>OB                 11
LOCATION_OF_PSloOB                6
PROCESS_OF_PSpro>OB               6
ISA_OBi>PS                        5
COEXISTS_WITH_PScwOB              3
PART_OF_OBpo>PS                   3
ADMINISTERED_TO_OBatPS            2
STIMULATES_OBstPS                 2
ISA_PSi>OB                        1
PART_OF_PSpo>OB                   1
NEG_ASSOCIATED_WITH_OBnawPS       1
Name: TYPE, dtype: int64


In [150]:
combo_num = 100

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Objects --- Procedures 

USES_PRuOB                   9720
compared_with_PRcpwOB         442
LOCATION_OF_OBloPR            177
NEG_USES_PRnuOB               166
higher_than_OBhtPR             88
METHOD_OF_OBmoPR               79
ISA_OBi>PR                     37
same_as_PRsaOB                 29
AFFECTS_OBafPR                 23
PRECEDES_OBpc>PR               17
lower_than_PRltOB              17
TREATS_OBtPR                   16
COEXISTS_WITH_PRcwOB           16
PRECEDES_PRpc>OB                9
ISA_PRi>OB                      8
NEG_higher_than_PRnhtOB         7
AUGMENTS_OBagPR                 6
NEG_AFFECTS_OBnafPR             4
CAUSES_OBcPR                    4
DIAGNOSES_PRdgOB                2
NEG_LOCATION_OF_OBnloPR         2
different_from_OBdfPR           2
NEG_CAUSES_OBncPR               2
STIMULATES_OBstPR               1
NEG_PRECEDES_OBnpcPR            1
PART_OF_OBpo>PR                 1
NEG_COEXISTS_WITH_PRncwOB       1
ASSOCIATED_WITH_OBawPR          1
Name: TYPE, dtype: int6

In [151]:
combo_num = 101

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Occupations --- Occupations 

ISA_OCi>OC               1435
METHOD_OF_OCmoOC           16
compared_with_OCcpwOC       1
higher_than_OChtOC          1
Name: TYPE, dtype: int64


In [152]:
combo_num = 102

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Occupations --- Organizations 

LOCATION_OF_ORloOC    67
METHOD_OF_ORmoOC       4
Name: TYPE, dtype: int64


In [153]:
combo_num = 103

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Occupations --- Phenomena 

METHOD_OF_PHmoOC        15
AFFECTS_PHafOC          12
PROCESS_OF_OCpro>PH      9
COEXISTS_WITH_OCcwPH     3
TREATS_OCtPH             2
NEG_CAUSES_OCncPH        1
PROCESS_OF_PHpro>OC      1
PREVENTS_OCpvPH          1
NEG_AFFECTS_PHnafOC      1
Name: TYPE, dtype: int64


In [154]:
combo_num = 104

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Occupations --- Physiology 

AFFECTS_PSafOC               72
COEXISTS_WITH_OCcwPS         38
METHOD_OF_PSmoOC             34
CAUSES_OCcPS                 23
PROCESS_OF_PSpro>OC          15
PROCESS_OF_OCpro>PS          10
TREATS_OCtPS                  6
ISA_OCi>PS                    6
ISA_PSi>OC                    5
DISRUPTS_PSdsOC               5
NEG_COEXISTS_WITH_OCncwPS     2
PRECEDES_PSpc>OC              2
NEG_CAUSES_OCncPS             1
AUGMENTS_OCagPS               1
Name: TYPE, dtype: int64


In [155]:
combo_num = 105

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Occupations --- Procedures 

METHOD_OF_PRmoOC             14437
COEXISTS_WITH_PRcwOC           256
NEG_METHOD_OF_PRnmoOC          192
PRECEDES_OCpc>PR                84
TREATS_PRtOC                    50
ADMINISTERED_TO_PRatOC          46
PRECEDES_PRpc>OC                26
USES_OCuPR                      26
AFFECTS_PRafOC                  22
ISA_PRi>OC                      22
compared_with_PRcpwOC           21
DIAGNOSES_PRdgOC                18
ISA_OCi>PR                      15
NEG_COEXISTS_WITH_PRncwOC        4
NEG_PRECEDES_OCnpcPR             3
higher_than_PRhtOC               3
STIMULATES_PRstOC                3
NEG_DIAGNOSES_PRndgOC            2
same_as_OCsaPR                   2
lower_than_OCltPR                1
PREVENTS_PRpvOC                  1
NEG_USES_OCnuPR                  1
NEG_AFFECTS_PRnafOC              1
NEG_PREVENTS_PRnpvOC             1
Name: TYPE, dtype: int64


In [156]:
combo_num = 106

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Organizations --- Organizations 

ISA_ORi>OR            606
LOCATION_OF_ORloOR      7
Name: TYPE, dtype: int64


In [157]:
combo_num = 107

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Organizations --- Phenomena 

LOCATION_OF_ORloPH    78
ISA_ORi>PH             2
ISA_PHi>OR             1
Name: TYPE, dtype: int64


In [158]:
combo_num = 108

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Organizations --- Physiology 

LOCATION_OF_ORloPS         95
PROCESS_OF_PSpro>OR         7
USES_PSuOR                  3
COEXISTS_WITH_ORcwPS        2
NEG_LOCATION_OF_ORnloPS     1
AFFECTS_ORafPS              1
DISRUPTS_ORdsPS             1
NEG_AFFECTS_ORnafPS         1
TREATS_ORtPS                1
Name: TYPE, dtype: int64


In [159]:
combo_num = 109

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Organizations --- Procedures 

LOCATION_OF_ORloPR         31645
NEG_LOCATION_OF_ORnloPR      410
USES_PRuOR                   155
METHOD_OF_PRmoOR              25
TREATS_PRtOR                  12
COEXISTS_WITH_PRcwOR           4
NEG_USES_PRnuOR                3
ADMINISTERED_TO_PRatOR         2
compared_with_PRcpwOR          1
PRECEDES_PRpc>OR               1
PRODUCES_ORpd>PR               1
NEG_TREATS_PRntOR              1
Name: TYPE, dtype: int64


In [160]:
# Merge these on Location_of
to_merge = ['METHOD_OF_PRmoOR','USES_PRuOR']
merge_edge_types(to_merge, 'LOCATION_OF_ORloPR', swap=True)

In [161]:
combo_num = 110

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Phenomena --- Phenomena 

PROCESS_OF_PHpro>PH              7416
AFFECTS_PHafPH                   4907
ISA_PHi>PH                        823
NEG_AFFECTS_PHnafPH               307
COEXISTS_WITH_PHcwPH              270
NEG_PROCESS_OF_PHnproPH           195
NEG_COEXISTS_WITH_PHncwPH          32
MANIFESTATION_OF_PHmfo>PH          13
AUGMENTS_PHagPH                     9
DISRUPTS_PHdsPH                     5
CAUSES_PHcPH                        4
compared_with_PHcpwPH               2
METHOD_OF_PHmoPH                    2
PREDISPOSES_PHpsPH                  2
higher_than_PHhtPH                  1
NEG_DISRUPTS_PHndsPH                1
NEG_MANIFESTATION_OF_PHnmfoPH       1
TREATS_PHtPH                        1
ASSOCIATED_WITH_PHawPH              1
NEG_PREDISPOSES_PHnpsPH             1
Name: TYPE, dtype: int64


In [162]:
combo_num = 111

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Phenomena --- Physiology 

AFFECTS_PHafPS                   15858
PROCESS_OF_PSpro>PH               8443
PROCESS_OF_PHpro>PS               4134
NEG_AFFECTS_PHnafPS               1358
COEXISTS_WITH_PScwPH              1053
NEG_PROCESS_OF_PSnproPH            336
CAUSES_PScPH                       178
MANIFESTATION_OF_PHmfo>PS          166
PRECEDES_PSpc>PH                    70
ISA_PSi>PH                          62
AUGMENTS_PHagPS                     59
NEG_COEXISTS_WITH_PHncwPS           58
DISRUPTS_PHdsPS                     50
ISA_PHi>PS                          45
PRECEDES_PHpc>PS                    21
MANIFESTATION_OF_PSmfo>PH           10
OCCURS_IN_PHoiPS                    10
ASSOCIATED_WITH_PSawPH               9
PREDISPOSES_PHpsPS                   9
TREATS_PStPH                         6
NEG_MANIFESTATION_OF_PHnmfoPS        6
NEG_CAUSES_PSncPH                    6
DIAGNOSES_PSdgPH                     4
NEG_DISRUPTS_PHndsPS                 3
NEG_AUGMENTS_PHnagPS                 

In [163]:
to_merge = ['AFFECTS_PHafPS',  'PROCESS_OF_PHpro>PS']
merge_edge_types(to_merge, 'AFFECTS_PHafPS')

change_edge_type('PROCESS_OF_PSpro>PH', 'AFFECTS_PHafPS', swap=True)

In [164]:
combo_num = 112

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Phenomena --- Procedures 

TREATS_PRtPH                 1383
AFFECTS_PRafPH               1193
DIAGNOSES_PRdgPH              577
METHOD_OF_PHmoPR              430
USES_PHuPR                    204
PRODUCES_PRpd>PH              164
PROCESS_OF_PHpro>PR           128
NEG_AFFECTS_PRnafPH           122
PROCESS_OF_PRpro>PH           100
STIMULATES_PRstPH              71
NEG_TREATS_PRntPH              68
PREVENTS_PRpvPH                56
COEXISTS_WITH_PRcwPH           28
PRECEDES_PRpc>PH               22
NEG_DIAGNOSES_PRndgPH          16
CAUSES_PRcPH                   13
LOCATION_OF_PHloPR              9
NEG_METHOD_OF_PHnmoPR           8
COMPLICATES_PRcpPH              7
PREDISPOSES_PRpsPH              6
NEG_USES_PRnuPH                 6
AUGMENTS_PRagPH                 6
compared_with_PHcpwPR           6
ISA_PRi>PH                      6
NEG_PROCESS_OF_PHnproPR         5
PRECEDES_PHpc>PR                4
INHIBITS_PHinPR                 2
NEG_STIMULATES_PRnstPH          2
DISRUPTS_PRdsPH      

In [165]:
combo_num = 113

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Physiology --- Physiology 

AFFECTS_PSafPS                   56131
COEXISTS_WITH_PScwPS             45656
PROCESS_OF_PSpro>PS              18340
CAUSES_PScPS                     11676
ISA_PSi>PS                        6247
PRECEDES_PSpc>PS                  4853
NEG_AFFECTS_PSnafPS               4598
NEG_COEXISTS_WITH_PSncwPS         1679
DISRUPTS_PSdsPS                   1422
MANIFESTATION_OF_PSmfo>PS          926
NEG_PROCESS_OF_PSnproPS            740
NEG_CAUSES_PSncPS                  714
NEG_DISRUPTS_PSndsPS                83
AUGMENTS_PSagPS                     70
PREDISPOSES_PSpsPS                  45
ASSOCIATED_WITH_PSawPS              39
NEG_PRECEDES_PSnpcPS                31
NEG_MANIFESTATION_OF_PSnmfoPS       16
OCCURS_IN_PSoiPS                    15
INTERACTS_WITH_PSiwPS               13
PRODUCES_PSpd>PS                    12
DIAGNOSES_PSdgPS                    10
TREATS_PStPS                         8
compared_with_PScpwPS                7
INHIBITS_PSinPS                     

In [166]:
# Just merge the tops to a general assocation
to_merge = ['AFFECTS_PSafPS', 'CAUSES_PScPS', 'PROCESS_OF_PSpro>PS', 'COEXISTS_WITH_PScwPS', 'DISRUPTS_PSdsPS']
merge_edge_types(to_merge, 'AFFECTS_PSafPS')

In [167]:
combo_num = 114

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Physiology --- Procedures 

AFFECTS_PRafPS                 31765
TREATS_PRtPS                   10764
STIMULATES_PRstPS               4891
NEG_AFFECTS_PRnafPS             4183
DIAGNOSES_PRdgPS                 730
NEG_TREATS_PRntPS                592
METHOD_OF_PRmoPS                 558
COEXISTS_WITH_PScwPR             543
USES_PRuPS                       430
PRECEDES_PRpc>PS                 366
NEG_STIMULATES_PRnstPS           356
PROCESS_OF_PSpro>PR              309
CAUSES_PRcPS                     280
COMPLICATES_PRcpPS               236
PREVENTS_PRpvPS                  189
ADMINISTERED_TO_PRatPS           155
PROCESS_OF_PRpro>PS              148
DISRUPTS_PRdsPS                   99
PREDISPOSES_PRpsPS                79
PRECEDES_PSpc>PR                  52
NEG_PROCESS_OF_PSnproPR           50
NEG_DIAGNOSES_PRndgPS             46
NEG_COEXISTS_WITH_PRncwPS         41
AUGMENTS_PRagPS                   40
ISA_PRi>PS                        35
LOCATION_OF_PSloPR                25
NEG_COMPLI

In [168]:
to_merge = ['AFFECTS_PRafPS', 'TREATS_PRtPS', 'STIMULATES_PRstPS']
merge_edge_types(to_merge, 'AFFECTS_PRafPS')

In [169]:
combo_num = 115

pair = combos[combo_num]

print(pair[0], '---', pair[1], '\n')
print(combo_counts[combos[combo_num]])

Procedures --- Procedures 

METHOD_OF_PRmoPR               163645
PRECEDES_PRpc>PR                58615
COEXISTS_WITH_PRcwPR            27740
compared_with_PRcpwPR           26839
USES_PRuPR                      26415
ISA_PRi>PR                      20017
higher_than_PRhtPR               7089
NEG_METHOD_OF_PRnmoPR            4677
AFFECTS_PRafPR                   2991
same_as_PRsaPR                   1915
lower_than_PRltPR                1733
NEG_PRECEDES_PRnpcPR             1130
TREATS_PRtPR                      906
NEG_COEXISTS_WITH_PRncwPR         671
NEG_higher_than_PRnhtPR           600
NEG_USES_PRnuPR                   566
DIAGNOSES_PRdgPR                  272
NEG_AFFECTS_PRnafPR               243
LOCATION_OF_PRloPR                157
different_from_PRdfPR             150
NEG_lower_than_PRnltPR             73
NEG_TREATS_PRntPR                  52
different_than_PRdtPR              47
ADMINISTERED_TO_PRatPR             46
CAUSES_PRcPR                       28
PREVENTS_PRpvPR       

In [170]:
to_merge = ['METHOD_OF_PRmoPR', 'USES_PRuPR',  'ISA_PRi>PR', 'COEXISTS_WITH_PRcwPR']
merge_edge_types(to_merge, 'METHOD_OF_PRmoPR')

In [171]:
edges['TYPE'].nunique()

2874

In [172]:
print(len(edges))

# sort by pmid and keep the first so the highest number of counts is retained
edges = edges.sort_values('pmids', ascending=False)
edges = edges.drop_duplicates(subset=['START_ID', 'END_ID', 'TYPE'], keep='first').reset_index(drop=True)

print(len(edges))

19857710
17730091


In [173]:
# Sort values before writing to disk
nodes = nodes.sort_values('LABEL')
edges = edges.sort_values('TYPE')

# Add in colons required by neo4j
nodes = nodes.rename(columns={'ID': ':ID', 'LABEL': ':LABEL'})
edges = edges.rename(columns={'START_ID': ':START_ID', 'END_ID': ':END_ID', 'TYPE': ':TYPE'})

nodes.to_csv('../data/nodes_condesed.csv', index=False)
edges.to_csv('../data/edges_condesed.csv', index=False)